In [12]:
import pandas as pd
import datetime
from decimal import Decimal
from django.db import transaction

# from_date = '2022-08-01'
# to_date = '2022-10-10'
# date_range = pd.date_range(start=from_date, end=to_date)
sid = transaction.savepoint()
try:
        sale_date="2022-08-01"
#         sale_date = str(date)[:10]
#         date_in_format = datetime.datetime.strptime(sale_date, '%Y-%m-%d')
#         previous_date =  date_in_format - datetime.timedelta(days=1) 
#         print(date_in_format)
#         GoodsReceiptRecordForDaily.objects.filter(sale_date=sale_date).delete()
        daily_sale_records = GoodsReceiptRecordForDaily.objects.filter(sale_date=sale_date).exclude(quantity_now=0)
        for daily_sale in daily_sale_records:
            daily_sale_record_obj = GoodsReceiptRecordForDaily(sale_date=sale_date,
                                                               by_product_id=daily_sale.by_product_id,
                                                               grn_number=daily_sale.grn_number,
                                                               quantity_at_receipt=daily_sale.quantity_now,
                                                               quantity_now=daily_sale.quantity_now,
                                                               quantity_now_time=datetime.datetime.now(),
                                                               price_per_unit=daily_sale.price_per_unit,
                                                               total_price=daily_sale.total_price,
                                                               )
            daily_sale_record_obj.save()
            print('old sale updated')
        if GoodsReceiptRecord.objects.filter(goods_receipt_master__grn_date__date=sale_date).exists():
            print('goods receipt available')
            goods_receipt_record_objs = GoodsReceiptRecord.objects.filter(goods_receipt_master__grn_date__date=sale_date)
            for goods_receipt_record_obj in goods_receipt_record_objs:
                daily_goods_receipt_record_obj = GoodsReceiptRecordForDaily(sale_date=sale_date,
                                                              by_product_id=goods_receipt_record_obj.by_product.id,
                                                              quantity_at_receipt=goods_receipt_record_obj.quantity_at_receipt,
                                                              quantity_now=goods_receipt_record_obj.quantity_at_receipt,
                                                              quantity_now_time=goods_receipt_record_obj.quantity_now_time,
                                                              price_per_unit=goods_receipt_record_obj.price_per_unit,
                                                              total_price=goods_receipt_record_obj.total_price,
                                                              grn_number=goods_receipt_record_obj.goods_receipt_master.grn_number,
                                                              )
                daily_goods_receipt_record_obj.save()
        by_sale_objs = BySale.objects.filter(by_sale_group__ordered_date=sale_date)
        for sale in by_sale_objs:
            print(sale.count)
            daily_goods_receipt_record_obj = GoodsReceiptRecordForDaily.objects.filter(by_product_id=sale.by_product.id, sale_date=sale_date).exclude(quantity_now=0).order_by('id')
            required_qty = sale.count
            for goods in daily_goods_receipt_record_obj:
                if goods.quantity_now >= required_qty:
                    balance_in_goods = goods.quantity_now - Decimal(required_qty)
                    goods.quantity_now = balance_in_goods
                    goods.quantity_now_time = datetime.datetime.now()
                    goods.save()

                    goods_receipt_record_sale_map_obj = DailyGoodsReceiptRecordBySaleMap(
                        daily_goods_receipt_record_id=goods.id,
                        by_sale_id=sale.id,
                        quantity_dispatched=required_qty
                    )
                    goods_receipt_record_sale_map_obj.save()
                    break
                else:
                    required_qty = Decimal(required_qty) - goods.quantity_now
                    goods_receipt_record_sale_map_obj = DailyGoodsReceiptRecordBySaleMap(
                        daily_goods_receipt_record_id=goods.id,
                        by_sale_id=sale.id,
                        quantity_dispatched=goods.quantity_now
                    )
                    goods_receipt_record_sale_map_obj.save()

                    goods.quantity_now = 0
                    goods.quantity_now_time = datetime.datetime.now()
                    goods.save()
        transaction.savepoint_commit(sid)
    
except Exception as e:
    print('Error on {}'.format(e))
    transaction.savepoint_rollback(sid)

/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField GoodsReceiptRecordForDaily.quantity_now_time received a naive datetime (2022-10-17 09:47:51.843458) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField GoodsReceiptRecordForDaily.quantity_now_time received a naive datetime (2022-10-17 09:47:51.846083) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField GoodsReceiptRecordForDaily.quantity_now_time received a naive datetime (2022-10-17 09:47:51.848098) while time zone support is active.
  warnings.warn("DateTimeField %s rec

old sale updated
old sale updated
old sale updated
old sale updated
old sale updated
goods receipt available
10.0
40.0
3.0
20.0
40.0
4.0
50.0
50.0
5.0
10.0
240.0
270.0
10.0
10.0
50.0
100.0
2000.0
500.0
10.0
3.0
50.0
50.0
5.0
10.0
5.0
4.0
5.0
2.0
5.0
20.0
40.0
40.0
5.0
30.0
60.0
120.0
270.0
20.0
2.0
30.0
40.0
20.0
135.0
14.0
20.0
20.0
20.0
5.0
20.0
20.0
5.0
10.0
120.0
135.0
36.0
54.0
60.0
80.0
80.0
150.0
60.0
50.0
60.0
100.0
70.0
5.0
100.0
120.0
100.0
30.0
20.0
5.0
10.0
15.0
2.0
3.0
20.0
10.0


/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField GoodsReceiptRecordForDaily.quantity_now_time received a naive datetime (2022-10-17 09:47:52.053877) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField GoodsReceiptRecordForDaily.quantity_now_time received a naive datetime (2022-10-17 09:47:52.064006) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField GoodsReceiptRecordForDaily.quantity_now_time received a naive datetime (2022-10-17 09:47:52.090258) while time zone support is active.
  warnings.warn("DateTimeField %s rec

In [1]:
BankTransactionMaster.objects.filter(id=139).delete()

(18663,
 {'transaction.BankTransactionCarryForward': 524,
  'transaction.BankTransactionPaymentResponseMap': 9068,
  'transaction.BankTransaction': 9070,
  'transaction.BankTransactionMaster': 1})

In [2]:
BankTransactionMaster.objects.filter(id=140).delete()

(1,
 {'transaction.BankTransactionCarryForward': 0,
  'transaction.BankTransactionMaster': 1})

In [8]:
BankTransactionMaster.objects.filter(id=134).delete()

InterfaceError: connection already closed

In [4]:
BankTransactionMaster.objects.filter(id=135).delete()

(1,
 {'transaction.BankTransactionCarryForward': 0,
  'transaction.BankTransactionMaster': 1})

In [5]:
BankTransactionMaster.objects.filter(id=138).delete()

(1,
 {'transaction.BankTransactionCarryForward': 0,
  'transaction.BankTransactionMaster': 1})

In [6]:
BankTransactionMaster.objects.filter(id=141).delete()

(9333,
 {'transaction.BankTransactionCarryForward': 1,
  'transaction.BankTransaction': 9331,
  'transaction.BankTransactionMaster': 1})

In [1]:
BankTransactionMaster.objects.filter(id=134).delete()

(18658,
 {'transaction.BankTransactionCarryForward': 9328,
  'transaction.BankTransactionPaymentResponseMap': 9329,
  'transaction.BankTransactionMaster': 1})

In [2]:
BankTransactionMaster.objects.filter(id=135).delete()

(1,
 {'transaction.BankTransactionCarryForward': 0,
  'transaction.BankTransactionMaster': 1})

In [3]:
BankTransactionMaster.objects.filter(id=138).delete()

(9333,
 {'transaction.BankTransactionCarryForward': 1,
  'transaction.BankTransaction': 9331,
  'transaction.BankTransactionMaster': 1})

In [4]:
BankTransactionMaster.objects.filter(id=142).delete()

(19182,
 {'transaction.BankTransactionCarryForward': 521,
  'transaction.BankTransactionPaymentResponseMap': 9329,
  'transaction.BankTransaction': 9331,
  'transaction.BankTransactionMaster': 1})

In [6]:
BankTransactionMaster.objects.filter(id=141)

<QuerySet [<BankTransactionMaster: BankTransactionMaster object>]>

In [7]:
BankTransactionMaster.objects.filter(id=139).delete()

(18663,
 {'transaction.BankTransactionCarryForward': 524,
  'transaction.BankTransactionPaymentResponseMap': 9068,
  'transaction.BankTransaction': 9070,
  'transaction.BankTransactionMaster': 1})

In [8]:
BankTransactionMaster.objects.filter(id=141).delete()

(39,
 {'transaction.BankTransactionCarryForward': 0,
  'transaction.BankTransactionPaymentResponseMap': 19,
  'transaction.BankTransaction': 19,
  'transaction.BankTransactionMaster': 1})

In [12]:
BankTransactionMaster.objects.filter(id=134).delete()

(0, {})

In [10]:
BankTransactionMaster.objects.filter(id=135).delete()

(0, {})

In [11]:
BankTransactionMaster.objects.filter(id=138).delete()

(0, {})

In [132]:
# @api_view(['POST'])
# def serve_daily_cash_and_credit_statement_excel(request): 
# print(request.data)
import pandas as pd
import datetime
import os
import numpy as np
date_from = "2022-10-01"
date_to = "2022-10-02"
if BySale.objects.filter(by_sale_group__ordered_date__range=[date_from, date_to]).exists():
    main_sale_cash_obj = BySale.objects.filter(by_sale_group__ordered_date__range=[date_from, date_to], by_sale_group__payment_method_id=1)
    main_sale_cash_list = list(main_sale_cash_obj.values_list('by_product', 'by_product__by_product_group_id', 'by_product__by_product_group__name', 'cost', 'cgst_value', 'sgst_value', 'total_cost', 'by_sale_group__ordered_date', 'by_sale_group__payment_method_id'))
    main_sale_cash_column = ['by_product_id', 'group_id', 'group_name', 'cost', 'cgst_value', 'sgst_value', 'cash_total_cost', 'ordered_date', 'payment_method']
    main_sale_cash_df = pd.DataFrame(main_sale_cash_list, columns=main_sale_cash_column)

    main_sale_credit_obj = BySale.objects.filter(by_sale_group__ordered_date__range=[date_from, date_to], by_sale_group__payment_method_id=2)
    main_sale_credit_list = list(main_sale_credit_obj.values_list('by_product', 'by_product__by_product_group_id', 'by_product__by_product_group__name', 'cost', 'cgst_value', 'sgst_value', 'total_cost', 'by_sale_group__ordered_date', 'by_sale_group__payment_method_id', 'by_sale_group__business_id'))
    main_sale_credit_column = ['by_product_id', 'group_id', 'group_name', 'cost', 'cgst_value', 'sgst_value', 'credit_total_cost', 'ordered_date', 'payment_method', 'business_id']
    main_sale_credit_df = pd.DataFrame(main_sale_credit_list, columns=main_sale_credit_column)
    
    cash_sale_group_wise_df = main_sale_cash_df.groupby('group_id').agg({'cost': sum})['cost']
    credit_sale_group_wise_df= main_sale_credit_df.groupby('group_id').agg({'cost': sum})['cost']
    
    
#     df.insert(0,"S NO", np.arange(1,len(df)+1))
    main_sale_group_cash_obj = BySaleGroup.objects.filter(ordered_date__range=[date_from, date_to], payment_method_id=1)

    main_sale_group_credit_obj = BySaleGroup.objects.filter(ordered_date__range=[date_from, date_to], payment_method_id=2)
    main_sale_group_credit_list = list(main_sale_group_credit_obj.values_list('id', 'business', 'total_cost'))
    main_sale_group_credit_column = ['id', 'business_id', 'total_cost']
    main_sale_group_credit_df = pd.DataFrame(main_sale_group_credit_list, columns=main_sale_group_credit_column)

    cgst_and_sgst_dict = {}
    if not main_sale_cash_df.empty:
        cgst_and_sgst_dict['cgst_collection_for_cash'] = main_sale_cash_df['cgst_value'].sum()
        cgst_and_sgst_dict['sgst_collection_for_cash'] = main_sale_cash_df['sgst_value'].sum()
        cgst_and_sgst_dict['total_cash_cost'] = main_sale_cash_df['cash_total_cost'].sum()
    else:
        cgst_and_sgst_dict['cgst_collection_for_cash'] = 0
        cgst_and_sgst_dict['sgst_collection_for_cash'] = 0
        cgst_and_sgst_dict['total_cash_cost'] = 0
    if not main_sale_credit_df.empty:
        cgst_and_sgst_dict['cgst_collection_for_credit'] = main_sale_credit_df['cgst_value'].sum()
        cgst_and_sgst_dict['sgst_collection_for_credit'] = main_sale_credit_df['sgst_value'].sum()
        cgst_and_sgst_dict['total_credit_cost'] = main_sale_credit_df['credit_total_cost'].sum()
    else:
        cgst_and_sgst_dict['cgst_collection_for_credit'] = 0
        cgst_and_sgst_dict['sgst_collection_for_credit'] = 0
        cgst_and_sgst_dict['total_credit_cost'] = 0
    cgst_and_sgst_dict['cgst_account_code'] = '30910'
    cgst_and_sgst_dict['sgst_account_code'] = '30911'
    cgst_and_sgst_dict['misc_account_code'] = '31004'
    if main_sale_group_cash_obj:
        cgst_and_sgst_dict['total_cash_amount'] = main_sale_group_cash_obj.aggregate(Sum('total_cost'))['total_cost__sum']
    else:
        cgst_and_sgst_dict['total_cash_amount'] = 0
    if main_sale_group_credit_obj:
        cgst_and_sgst_dict['total_credit_amount'] = main_sale_group_credit_obj.aggregate(Sum('total_cost'))['total_cost__sum']
    else:
        cgst_and_sgst_dict['total_credit_amount'] = 0

    product_group_obj = ByProductGroup.objects.filter()
    product_group_list = list(product_group_obj.values_list('id', 'name', 'account_code'))
    product_group_column = ['group_id', 'name', 'account_code']
    product_group_df = pd.DataFrame(product_group_list, columns=product_group_column)
    product_group_list = product_group_df.to_dict('r')

    df1 = pd.merge(cash_sale_group_wise_df, credit_sale_group_wise_df, left_on='group_id', right_on='group_id', how='left')
    
    df2 = pd.merge(df1,product_group_df,left_on='group_id', right_on='group_id', how='left')
    df2 = df2.reindex(columns=['account_code','name',"cash_total_cost",'credit_total_cost'])
        
    df2 = df2.rename(columns={'account_code':'Code', 'name':'Product',"cash_total_cost":'Cash','credit_total_cost':'Credit'})
    df2.insert(0,"S NO", np.arange(1,len(df2)+1))
    
    credit_business_ids = list(set(main_sale_credit_obj.values_list('by_sale_group__business_id', flat=True)))

    business_obj = Business.objects.filter(id__in=credit_business_ids)
    business_list = list(business_obj.values_list('id', 'businessagentmap__agent__agent_code', 'businessagentmap__agent__first_name'))
    business_column = ['business_id', 'account_code', 'agent_name']
    business_df = pd.DataFrame(business_list, columns=business_column)
    business_df = business_df.fillna(0)
    business_list = business_df.to_dict('r')

    credit_sale_business_wise_df = main_sale_group_credit_df.groupby('business_id').agg({'total_cost': sum})['total_cost']

    df = pd.merge(business_df, credit_sale_business_wise_df, left_on='business_id', right_on='business_id', how='left')
    df = df.reindex(columns=["account_code",'agent_name',"total_cost"])
    df[['account_code', 'agent_name', 'total_cost']].fillna('')    
    df = df.rename(columns={ 'account_code': 'Code','agent_name':'Debitors','total_cost':'Amount'})
    df.insert(0,"S NO", np.arange(1,len(df)+1))
#     df.fillna('')
    
#     frames = [df, df2,]

#     result = pd.concat(frames)
    # Excel Part
    from_date_in_strip = datetime.datetime.strptime(date_from, "%Y-%m-%d")
    from_date_in_format = datetime.datetime.strftime(from_date_in_strip, '%d-%m-%Y')

    to_date_in_strip = datetime.datetime.strptime(date_to, "%Y-%m-%d")
    to_date_in_format = datetime.datetime.strftime(to_date_in_strip, '%d-%m-%Y')

    file_name = f"{from_date_in_format} - {to_date_in_format} - daily_sales_statement.xlsx"
    file_path = os.path.join('static/media/by_product/daily_sale/')

    try:
        path = file_path
        os.makedirs(path)
    except FileExistsError:
        print('already created')
    
    file_path = os.path.join('static/media/by_product/daily_sale/', file_name)

already created


/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


In [37]:
main_sale_cash_df

,by_product_id,group_id,group_name,cost,cgst_value,sgst_value,total_cost,ordered_date,payment_method
0,474,84,Ghee,9221.4000,553.3000,553.3000,10328.0000,2022-10-01,1
1,469,83,Peda,906.8000,22.6000,22.6000,952.0000,2022-10-01,1
2,474,84,Ghee,23053.5000,1383.2500,1383.2500,25820.0000,2022-10-01,1
3,478,86,Badam Power,783.5000,70.5000,70.5000,924.5000,2022-10-01,1
4,492,97,Cookies,256.8000,23.1000,23.1000,303.0000,2022-10-01,1
5,506,102,Icecream,804.4000,72.0000,72.0000,948.4000,2022-10-01,1
6,507,102,Icecream,1608.8000,144.0000,144.0000,1896.8000,2022-10-01,1
7,508,102,Icecream,216.9000,19.5300,19.5300,255.9600,2022-10-01,1
8,516,102,Icecream,643.2000,58.0800,58.0800,759.3600,2022-10-01,1
9,518,102,Icecream,214.4000,19.3600,19.3600,253.1200,2022-10-01,1


In [38]:
main_sale_credit_df

,by_product_id,group_id,group_name,cost,cgst_value,sgst_value,total_cost,ordered_date,payment_method,business_id
0,506,102,Icecream,5088.0000,456.0000,456.0000,6000.0000,2022-10-01,2,1775
1,507,102,Icecream,2544.0000,228.0000,228.0000,3000.0000,2022-10-01,2,1775
2,504,102,Icecream,678.0000,61.0000,61.0000,800.0000,2022-10-01,2,1775
3,511,102,Icecream,335.5200,30.2400,30.2400,396.0000,2022-10-01,2,1775
4,512,102,Icecream,335.5200,30.2400,30.2400,396.0000,2022-10-01,2,1775
...,...,...,...,...,...,...,...,...,...,...
56,482,88,Mysorepa,666.7000,16.6500,16.6500,700.0000,2022-10-01,2,2434
57,572,88,Mysorepa,761.9000,19.0500,19.0500,800.0000,2022-10-01,2,2434
58,484,90,Kajukathri,946.7000,24.1500,24.1500,995.0000,2022-10-01,2,2434
59,485,91,Milk Shake,3640.9500,204.5250,204.5250,4050.0000,2022-10-01,2,1775


In [39]:
cash_sale_group_wise

{83: Decimal('1813.6000'),
 84: Decimal('131403.0000'),
 86: Decimal('783.5000'),
 87: Decimal('784.8000'),
 88: Decimal('923.5200'),
 90: Decimal('441.0000'),
 91: Decimal('2354.4000'),
 94: Decimal('749.2000'),
 97: Decimal('1555.2000'),
 102: Decimal('4301.9000')}

In [40]:
credit_sale_group_wise

{83: Decimal('48854.8000'),
 84: Decimal('346966.2000'),
 85: Decimal('1608.0000'),
 86: Decimal('17799.0000'),
 87: Decimal('17681.4000'),
 88: Decimal('8000.1600'),
 90: Decimal('5301.5200'),
 91: Decimal('25486.6500'),
 97: Decimal('17139.5000'),
 102: Decimal('24585.2000')}

In [41]:
main_sale_group_credit_df

,id,business_id,total_cost
0,2716,1775,20180.00
1,2718,2059,6420.00
2,2719,2060,2400.00
3,2729,1775,426100.00
4,2730,2060,41520.00
5,2731,2435,34017.00
6,2732,2059,14425.00
7,2734,1775,720.00
8,2733,2434,22415.00
9,2738,1775,5850.00


In [42]:
product_group_list

[{'id': 107, 'name': 'Drinking Yogurt', 'account_code': '0'},
 {'id': 101, 'name': 'Rose Milk', 'account_code': '30226'},
 {'id': 98, 'name': 'Milk Chocolate', 'account_code': '0.0'},
 {'id': 91, 'name': 'Milk Shake', 'account_code': '30231'},
 {'id': 87, 'name': 'F.L.M Bottle', 'account_code': '30203'},
 {'id': 84, 'name': 'Ghee', 'account_code': '30206'},
 {'id': 106, 'name': 'Dairy Whitener', 'account_code': '0'},
 {'id': 96, 'name': 'Gulabjamune', 'account_code': '30215'},
 {'id': 105, 'name': 'Tetra Pack', 'account_code': '0'},
 {'id': 94, 'name': 'Smp Powder', 'account_code': '30208'},
 {'id': 85, 'name': 'Butter', 'account_code': '30205'},
 {'id': 108, 'name': 'Drinking Noodles', 'account_code': '0'},
 {'id': 100, 'name': 'Gulabjamune Powder', 'account_code': '30215'},
 {'id': 99, 'name': 'Smp Bulk', 'account_code': '0.0'},
 {'id': 103, 'name': 'Milk Cake', 'account_code': '0'},
 {'id': 92, 'name': 'Curd', 'account_code': '30202'},
 {'id': 93, 'name': 'Butter Milk', 'account_cod

In [43]:
product_group_df

,id,name,account_code
0,107,Drinking Yogurt,0
1,101,Rose Milk,30226
2,98,Milk Chocolate,0.0
3,91,Milk Shake,30231
4,87,F.L.M Bottle,30203
5,84,Ghee,30206
6,106,Dairy Whitener,0
7,96,Gulabjamune,30215
8,105,Tetra Pack,0
9,94,Smp Powder,30208


In [12]:
business_df

,id,account_code,agent_name
0,1775,0,RSP UNION PARLOUR
1,2059,0,UKKADAM BUS STAND PARLOUR
2,2060,0,HI-TECH PARLOUR
3,2434,0,Gandhi Puram Bus Stand Parlour
4,2435,0,V.O.C.PARK PARLOUR


In [13]:
business_list

[{'id': 1775, 'account_code': 0, 'agent_name': 'RSP UNION PARLOUR'},
 {'id': 2059, 'account_code': 0, 'agent_name': 'UKKADAM BUS STAND PARLOUR'},
 {'id': 2060, 'account_code': 0, 'agent_name': 'HI-TECH PARLOUR'},
 {'id': 2434,
  'account_code': 0,
  'agent_name': 'Gandhi Puram Bus Stand Parlour'},
 {'id': 2435, 'account_code': 0, 'agent_name': 'V.O.C.PARK PARLOUR'}]

In [14]:
credit_sale_business_wise

{1775: Decimal('452850.00'),
 2059: Decimal('20845.00'),
 2060: Decimal('43920.00'),
 2434: Decimal('22415.00'),
 2435: Decimal('34017.00')}

In [20]:
credit_sale_business_wise_df

business_id
1775    452850.00
2059     20845.00
2060     43920.00
2434     22415.00
2435     34017.00
Name: total_cost, dtype: object

In [51]:
df

,S NO,Code,Debitors,Amount
0,1,8101,RSP UNION PARLOUR,452850.00
1,2,8104,UKKADAM BUS STAND PARLOUR,20845.00
2,3,8115,HI-TECH PARLOUR,43920.00
3,4,8105,Gandhi Puram Bus Stand Parlour,22415.00
4,5,8109,V.O.C.PARK PARLOUR,34017.00


In [53]:
df1

,Cash,Credit
group_id,,
83,NaN,NaN
84,NaN,NaN
86,NaN,NaN
87,NaN,NaN
88,NaN,NaN
90,NaN,NaN
91,NaN,NaN
94,NaN,NaN
97,NaN,NaN


In [62]:
df2

,S NO,Product,Code,Cash,Credit
0,1,Peda,30212,NaN,NaN
1,2,Ghee,30206,NaN,NaN
2,3,Badam Power,30216,NaN,NaN
3,4,F.L.M Bottle,30203,NaN,NaN
4,5,Mysorepa,30214,NaN,NaN
5,6,Kajukathri,30241,NaN,NaN
6,7,Milk Shake,30231,NaN,NaN
7,8,Smp Powder,30208,NaN,NaN
8,9,Cookies,30240,NaN,NaN
9,10,Icecream,30213,NaN,NaN


In [70]:
result

,S NO,Code,Debitors,Amount,Product,Cash,Credit
0,1,8101,RSP UNION PARLOUR,452850.00,NaN,NaN,NaN
1,2,8104,UKKADAM BUS STAND PARLOUR,20845.00,NaN,NaN,NaN
2,3,8115,HI-TECH PARLOUR,43920.00,NaN,NaN,NaN
3,4,8105,Gandhi Puram Bus Stand Parlour,22415.00,NaN,NaN,NaN
4,5,8109,V.O.C.PARK PARLOUR,34017.00,NaN,NaN,NaN
0,1,30212,NaN,NaN,Peda,NaN,NaN
1,2,30206,NaN,NaN,Ghee,NaN,NaN
2,3,30216,NaN,NaN,Badam Power,NaN,NaN
3,4,30203,NaN,NaN,F.L.M Bottle,NaN,NaN
4,5,30214,NaN,NaN,Mysorepa,NaN,NaN


In [20]:
import pandas as pd
import datetime
import os
import numpy as np
def serve_daily_cash_and_credit_statement_excel(request): 
    date_from = "2022-10-01"
    date_to = "2022-10-02"
    if BySale.objects.filter(by_sale_group__ordered_date__range=[date_from, date_to]).exists():
        main_sale_cash_obj = BySale.objects.filter(by_sale_group__ordered_date__range=[date_from, date_to], by_sale_group__payment_method_id=1)
        main_sale_cash_list = list(main_sale_cash_obj.values_list('by_product', 'by_product__by_product_group_id', 'by_product__by_product_group__name', 'cost', 'cgst_value', 'sgst_value', 'total_cost', 'by_sale_group__ordered_date', 'by_sale_group__payment_method_id'))
        main_sale_cash_column = ['by_product_id', 'group_id', 'group_name', 'cost', 'cgst_value', 'sgst_value', 'cash_total_cost', 'ordered_date', 'payment_method']
        main_sale_cash_df = pd.DataFrame(main_sale_cash_list, columns=main_sale_cash_column)

        main_sale_credit_obj = BySale.objects.filter(by_sale_group__ordered_date__range=[date_from, date_to], by_sale_group__payment_method_id=2)
        main_sale_credit_list = list(main_sale_credit_obj.values_list('by_product', 'by_product__by_product_group_id', 'by_product__by_product_group__name', 'cost', 'cgst_value', 'sgst_value', 'total_cost', 'by_sale_group__ordered_date', 'by_sale_group__payment_method_id', 'by_sale_group__business_id'))
        main_sale_credit_column = ['by_product_id', 'group_id', 'group_name', 'cost', 'cgst_value', 'sgst_value', 'credit_total_cost', 'ordered_date', 'payment_method', 'business_id']
        main_sale_credit_df = pd.DataFrame(main_sale_credit_list, columns=main_sale_credit_column)

        cash_sale_group_wise_df = main_sale_cash_df.groupby('group_id').agg({'cost': sum})['cost']
        credit_sale_group_wise_df= main_sale_credit_df.groupby('group_id').agg({'cost': sum})['cost']

        main_sale_group_cash_obj = BySaleGroup.objects.filter(ordered_date__range=[date_from, date_to], payment_method_id=1)

        main_sale_group_credit_obj = BySaleGroup.objects.filter(ordered_date__range=[date_from, date_to], payment_method_id=2)
        main_sale_group_credit_list = list(main_sale_group_credit_obj.values_list('id', 'business', 'total_cost'))
        main_sale_group_credit_column = ['id', 'business_id', 'total_cost']
        main_sale_group_credit_df = pd.DataFrame(main_sale_group_credit_list, columns=main_sale_group_credit_column)

        cgst_and_sgst_dict = {}
        if not main_sale_cash_df.empty:
            cgst_and_sgst_dict['cgst_collection_for_cash'] = main_sale_cash_df['cgst_value'].sum()
            cgst_and_sgst_dict['sgst_collection_for_cash'] = main_sale_cash_df['sgst_value'].sum()
            cgst_and_sgst_dict['total_cash_cost'] = main_sale_cash_df['cash_total_cost'].sum()
        else:
            cgst_and_sgst_dict['cgst_collection_for_cash'] = 0
            cgst_and_sgst_dict['sgst_collection_for_cash'] = 0
            cgst_and_sgst_dict['total_cash_cost'] = 0
        if not main_sale_credit_df.empty:
            cgst_and_sgst_dict['cgst_collection_for_credit'] = main_sale_credit_df['cgst_value'].sum()
            cgst_and_sgst_dict['sgst_collection_for_credit'] = main_sale_credit_df['sgst_value'].sum()
            cgst_and_sgst_dict['total_credit_cost'] = main_sale_credit_df['credit_total_cost'].sum()
        else:
            cgst_and_sgst_dict['cgst_collection_for_credit'] = 0
            cgst_and_sgst_dict['sgst_collection_for_credit'] = 0
            cgst_and_sgst_dict['total_credit_cost'] = 0
        cgst_and_sgst_dict['cgst_account_code'] = '30910'
        cgst_and_sgst_dict['sgst_account_code'] = '30911'
        cgst_and_sgst_dict['misc_account_code'] = '31004'
        if main_sale_group_cash_obj:
            cgst_and_sgst_dict['total_cash_amount'] = main_sale_group_cash_obj.aggregate(Sum('total_cost'))['total_cost__sum']
        else:
            cgst_and_sgst_dict['total_cash_amount'] = 0
        if main_sale_group_credit_obj:
            cgst_and_sgst_dict['total_credit_amount'] = main_sale_group_credit_obj.aggregate(Sum('total_cost'))['total_cost__sum']
        else:
            cgst_and_sgst_dict['total_credit_amount'] = 0

        product_group_obj = ByProductGroup.objects.filter()
        product_group_list = list(product_group_obj.values_list('id', 'name', 'account_code'))
        product_group_column = ['group_id', 'name', 'account_code']
        product_group_df = pd.DataFrame(product_group_list, columns=product_group_column)
        product_group_list = product_group_df.to_dict('r')

        df1 = pd.merge(cash_sale_group_wise_df, credit_sale_group_wise_df, left_on='group_id', right_on='group_id', how='left')
        merge_df = pd.merge(df1,product_group_df,left_on='group_id', right_on='group_id', how='left')

        merge_df = merge_df.reindex(columns=['account_code','name',"cost_x",'cost_y'])

        merge_df = merge_df.rename(columns={'account_code':'Code', 'name':'Product',"cost_x":'Cash','cost_y':'Credit'})
        merge_df = merge_df.append(merge_df[['Cash','Credit']].sum(), ignore_index=True)
        merge_df.iloc[-1, merge_df.columns.get_loc('Product')] = 'TOTAL'
        merge_df.insert(0,"S NO", np.arange(1,len(merge_df)+1))
        merge_df.fillna('')

        credit_business_ids = list(set(main_sale_credit_obj.values_list('by_sale_group__business_id', flat=True)))

        business_obj = Business.objects.filter(id__in=credit_business_ids)
        business_list = list(business_obj.values_list('id', 'businessagentmap__agent__agent_code', 'businessagentmap__agent__first_name'))
        business_column = ['business_id', 'account_code', 'agent_name']
        business_df = pd.DataFrame(business_list, columns=business_column)
        business_df = business_df.fillna(0)
        business_list = business_df.to_dict('r')

        credit_sale_business_wise_df = main_sale_group_credit_df.groupby('business_id').agg({'total_cost': sum})['total_cost']
        print(credit_sale_business_wise_df)
        df = pd.merge(business_df, credit_sale_business_wise_df, left_on='business_id', right_on='business_id', how='left')
        df = df.reindex(columns=["account_code",'agent_name',"total_cost"])

        df = df.rename(columns={ 'account_code': 'Code','agent_name':'Debitors','total_cost':'Amount'})
        df = df.append(df[['Amount']].sum(), ignore_index=True)
        df.iloc[-1, df.columns.get_loc('Debitors')] = 'TOTAL'
        df.insert(0,"S NO", np.arange(1,len(df)+1))
        df.fillna('')

        result_df = pd.concat([merge_df, df], axis=1)
        print(result_df)
        # Excel Part
        from_date_in_strip = datetime.datetime.strptime(date_from, "%Y-%m-%d")
        from_date_in_format = datetime.datetime.strftime(from_date_in_strip, '%d-%m-%Y')

        to_date_in_strip = datetime.datetime.strptime(date_to, "%Y-%m-%d")
        to_date_in_format = datetime.datetime.strftime(to_date_in_strip, '%d-%m-%Y')

        excel_file_name = f"{from_date_in_format} - {to_date_in_format} - daily_sales_statement.xlsx"
        excel_file_path = os.path.join('static/media/by_product/daily_sale/',excel_file_name)

        result_df.to_excel(excel_file_path)

#         try:
#             path = file_path
#             os.makedirs(path)
#         except FileExistsError:
#             print('already created')
#         document = {}
#         document['excel_file_name'] = excel_file_name


        try:
            image_path = excel_file_path
            with open(image_path, 'rb') as image_file:
                encoded_image = b64encode(image_file.read())
                document['excel'] = encoded_image
        except Exception as err:
            print(err)  

        return Response(data=document, status=status.HTTP_200_OK)
#  else:
#     document = {}
#     # document['file_name'] = file_name
#     document['is_data_available'] = False
#     return Response(data=document, status=status.HTTP_200_OK)

In [21]:
result_df

,S NO,Code,Product,Cash,Credit,S NO,Code,Debitors,Amount
0,1,30212,Peda,18071.0600,76738.0300,1,8101,RSP UNION PARLOUR,92831.00
1,2,30206,Ghee,359670.8600,780997.2000,2,8104,UKKADAM BUS STAND PARLOUR,10700.00
2,3,30205,Butter,19873.8300,35228.3200,3,8115,HI-TECH PARLOUR,82976.00
3,4,30216,Badam Power,2604.4300,10043.0000,4,8105,Gandhi Puram Bus Stand Parlour,35295.00
4,5,30203,F.L.M Bottle,2375.2800,19288.8000,5,8109,V.O.C.PARK PARLOUR,25350.00
5,6,30214,Mysorepa,2221.4100,17713.8000,6,54272,TNAU / NOFRC,2900.00
6,7,30244,Paneer,1422.4000,2489.2000,7,95432,Reliance Retail Limited,774560.00
7,8,30231,Milk Shake,11253.8700,18204.7500,8,4372,Samathur,32174.00
8,9,30208,Smp Powder,2097.9000,NaN,9,8124,RSP - HI TECH PARLOUR,39040.00
9,10,NaN,TOTAL,419591.0400,960703.1000,10,NaN,TOTAL,1095826.00


In [3]:
def serve_daily_cash_and_credit_statement_excel(request): 
    print(request.data)
    date_from =request.data['from_date']
    date_to = request.data['to_date']
    if BySale.objects.filter(by_sale_group__ordered_date__range=[date_from, date_to]).exists():
        main_sale_cash_obj = BySale.objects.filter(by_sale_group__ordered_date__range=[date_from, date_to], by_sale_group__payment_method_id=1)
        main_sale_cash_list = list(main_sale_cash_obj.values_list('by_product', 'by_product__by_product_group_id', 'by_product__by_product_group__name', 'cost', 'cgst_value', 'sgst_value', 'total_cost', 'by_sale_group__ordered_date', 'by_sale_group__payment_method_id'))
        main_sale_cash_column = ['by_product_id', 'group_id', 'group_name', 'cost', 'cgst_value', 'sgst_value', 'cash_total_cost', 'ordered_date', 'payment_method']
        main_sale_cash_df = pd.DataFrame(main_sale_cash_list, columns=main_sale_cash_column)

        main_sale_credit_obj = BySale.objects.filter(by_sale_group__ordered_date__range=[date_from, date_to], by_sale_group__payment_method_id=2)
        main_sale_credit_list = list(main_sale_credit_obj.values_list('by_product', 'by_product__by_product_group_id', 'by_product__by_product_group__name', 'cost', 'cgst_value', 'sgst_value', 'total_cost', 'by_sale_group__ordered_date', 'by_sale_group__payment_method_id', 'by_sale_group__business_id'))
        main_sale_credit_column = ['by_product_id', 'group_id', 'group_name', 'cost', 'cgst_value', 'sgst_value', 'credit_total_cost', 'ordered_date', 'payment_method', 'business_id']
        main_sale_credit_df = pd.DataFrame(main_sale_credit_list, columns=main_sale_credit_column)

        cash_sale_group_wise_df = main_sale_cash_df.groupby('group_id').agg({'cost': sum})['cost']
        credit_sale_group_wise_df= main_sale_credit_df.groupby('group_id').agg({'cost': sum})['cost']

        main_sale_group_cash_obj = BySaleGroup.objects.filter(ordered_date__range=[date_from, date_to], payment_method_id=1)

        main_sale_group_credit_obj = BySaleGroup.objects.filter(ordered_date__range=[date_from, date_to], payment_method_id=2)
        main_sale_group_credit_list = list(main_sale_group_credit_obj.values_list('id', 'business', 'total_cost'))
        main_sale_group_credit_column = ['id', 'business_id', 'total_cost']
        main_sale_group_credit_df = pd.DataFrame(main_sale_group_credit_list, columns=main_sale_group_credit_column)

        cgst_and_sgst_dict = {}
        if not main_sale_cash_df.empty:
            cgst_and_sgst_dict['cgst_collection_for_cash'] = main_sale_cash_df['cgst_value'].sum()
            cgst_and_sgst_dict['sgst_collection_for_cash'] = main_sale_cash_df['sgst_value'].sum()
            cgst_and_sgst_dict['total_cash_cost'] = main_sale_cash_df['cash_total_cost'].sum()
        else:
            cgst_and_sgst_dict['cgst_collection_for_cash'] = 0
            cgst_and_sgst_dict['sgst_collection_for_cash'] = 0
            cgst_and_sgst_dict['total_cash_cost'] = 0
        if not main_sale_credit_df.empty:
            cgst_and_sgst_dict['cgst_collection_for_credit'] = main_sale_credit_df['cgst_value'].sum()
            cgst_and_sgst_dict['sgst_collection_for_credit'] = main_sale_credit_df['sgst_value'].sum()
            cgst_and_sgst_dict['total_credit_cost'] = main_sale_credit_df['credit_total_cost'].sum()
        else:
            cgst_and_sgst_dict['cgst_collection_for_credit'] = 0
            cgst_and_sgst_dict['sgst_collection_for_credit'] = 0
            cgst_and_sgst_dict['total_credit_cost'] = 0
        cgst_and_sgst_dict['cgst_account_code'] = '30910'
        cgst_and_sgst_dict['sgst_account_code'] = '30911'
        cgst_and_sgst_dict['misc_account_code'] = '31004'
        if main_sale_group_cash_obj:
            cgst_and_sgst_dict['total_cash_amount'] = main_sale_group_cash_obj.aggregate(Sum('total_cost'))['total_cost__sum']
        else:
            cgst_and_sgst_dict['total_cash_amount'] = 0
        if main_sale_group_credit_obj:
            cgst_and_sgst_dict['total_credit_amount'] = main_sale_group_credit_obj.aggregate(Sum('total_cost'))['total_cost__sum']
        else:
            cgst_and_sgst_dict['total_credit_amount'] = 0

        product_group_obj = ByProductGroup.objects.filter()
        product_group_list = list(product_group_obj.values_list('id', 'name', 'account_code'))
        product_group_column = ['group_id', 'name', 'account_code']
        product_group_df = pd.DataFrame(product_group_list, columns=product_group_column)
        product_group_list = product_group_df.to_dict('r')

        df1 = pd.merge(cash_sale_group_wise_df, credit_sale_group_wise_df, left_on='group_id', right_on='group_id', how='left')
        merge_df = pd.merge(df1,product_group_df,left_on='group_id', right_on='group_id', how='left')

        merge_df = merge_df.reindex(columns=['account_code','name',"cost_x",'cost_y'])

        merge_df = merge_df.rename(columns={'account_code':'Code', 'name':'Product',"cost_x":'Cash','cost_y':'Credit'})
        merge_df = merge_df.append(merge_df[['Cash','Credit']].sum(), ignore_index=True)
        merge_df.iloc[-1, merge_df.columns.get_loc('Product')] = 'TOTAL'
        merge_df.insert(0,"S NO", np.arange(1,len(merge_df)+1))
        merge_df.fillna('')

        credit_business_ids = list(set(main_sale_credit_obj.values_list('by_sale_group__business_id', flat=True)))

        business_obj = Business.objects.filter(id__in=credit_business_ids)
        business_list = list(business_obj.values_list('id', 'businessagentmap__agent__agent_code', 'businessagentmap__agent__first_name'))
        business_column = ['business_id', 'account_code', 'agent_name']
        business_df = pd.DataFrame(business_list, columns=business_column)
        business_df = business_df.fillna(0)
        business_list = business_df.to_dict('r')

        credit_sale_business_wise_df = main_sale_group_credit_df.groupby('business_id').agg({'total_cost': sum})['total_cost']

        df = pd.merge(business_df, credit_sale_business_wise_df, left_on='business_id', right_on='business_id', how='left')
        df = df.reindex(columns=["account_code",'agent_name',"total_cost"])

        df = df.rename(columns={ 'account_code': 'Code','agent_name':'Debitors','total_cost':'Amount'})
        df = df.append(df[['Amount']].sum(), ignore_index=True)
        df.iloc[-1, df.columns.get_loc('Debitors')] = 'TOTAL'
        df.insert(0,"S NO", np.arange(1,len(df)+1))
        df.fillna('')

        result_df = pd.concat([merge_df, df], axis=1)
        # Excel Part
        from_date_in_strip = datetime.datetime.strptime(date_from, "%Y-%m-%d")
        from_date_in_format = datetime.datetime.strftime(from_date_in_strip, '%d-%m-%Y')

        to_date_in_strip = datetime.datetime.strptime(date_to, "%Y-%m-%d")
        to_date_in_format = datetime.datetime.strftime(to_date_in_strip, '%d-%m-%Y')

        excel_file_name = f"{from_date_in_format} - {to_date_in_format} - daily_sales_statement.xlsx"
        excel_file_path = os.path.join('static/media/by_product/daily_sale/',excel_file_name)

        result_df.to_excel(excel_file_path)

        try:
            path = excel_file_path
            os.makedirs(path)
        except FileExistsError:
            print('already created')
        document = {}
        document['excel_file_name'] = excel_file_name

        try:
            image_path = excel_file_path
            with open(image_path, 'rb') as image_file:
                encoded_image = b64encode(image_file.read())
                document['excel'] = encoded_image
        except Exception as err:
            print(err)  

        return Response(data=document, status=status.HTTP_200_OK)
#     else:
#         document = {}
#         document['is_data_available'] = False
#         return Response(data=document, status=status.HTTP_200_OK)


In [2]:
result_df

NameError: name 'result_df' is not defined

In [101]:
def serve_daily_cash_and_credit_statement_excel(request): 
# print(request.data)
    import pandas as pd
    import datetime
    import os
    import numpy as np
    from base64 import b64encode, b64decode
    date_from ="2022-10-01"
    date_to = "2022-10-10"
    if BySale.objects.filter(by_sale_group__ordered_date__range=[date_from, date_to]).exists():
        main_sale_cash_obj = BySale.objects.filter(by_sale_group__ordered_date__range=[date_from, date_to], by_sale_group__payment_method_id=1)
        main_sale_cash_list = list(main_sale_cash_obj.values_list('by_product', 'by_product__by_product_group_id', 'by_product__by_product_group__name', 'cost', 'cgst_value', 'sgst_value', 'total_cost', 'by_sale_group__ordered_date', 'by_sale_group__payment_method_id'))
        main_sale_cash_column = ['by_product_id', 'group_id', 'group_name', 'cost', 'cgst_value', 'sgst_value', 'cash_total_cost', 'ordered_date', 'payment_method']
        main_sale_cash_df = pd.DataFrame(main_sale_cash_list, columns=main_sale_cash_column)

        main_sale_credit_obj = BySale.objects.filter(by_sale_group__ordered_date__range=[date_from, date_to], by_sale_group__payment_method_id=2)
        main_sale_credit_list = list(main_sale_credit_obj.values_list('by_product', 'by_product__by_product_group_id', 'by_product__by_product_group__name', 'cost', 'cgst_value', 'sgst_value', 'total_cost', 'by_sale_group__ordered_date', 'by_sale_group__payment_method_id', 'by_sale_group__business_id'))
        main_sale_credit_column = ['by_product_id', 'group_id', 'group_name', 'cost', 'cgst_value', 'sgst_value', 'credit_total_cost', 'ordered_date', 'payment_method', 'business_id']
        main_sale_credit_df = pd.DataFrame(main_sale_credit_list, columns=main_sale_credit_column)

        cash_sale_group_wise_df = main_sale_cash_df.groupby('group_id').agg({'cost': sum})['cost']
        credit_sale_group_wise_df= main_sale_credit_df.groupby('group_id').agg({'cost': sum})['cost']

        main_sale_group_cash_obj = BySaleGroup.objects.filter(ordered_date__range=[date_from, date_to], payment_method_id=1)

        main_sale_group_credit_obj = BySaleGroup.objects.filter(ordered_date__range=[date_from, date_to], payment_method_id=2)
        main_sale_group_credit_list = list(main_sale_group_credit_obj.values_list('id', 'business', 'total_cost'))
        main_sale_group_credit_column = ['id', 'business_id', 'total_cost']
        main_sale_group_credit_df = pd.DataFrame(main_sale_group_credit_list, columns=main_sale_group_credit_column)

        cgst_and_sgst_dict = {}
        if not main_sale_cash_df.empty:
            cgst_and_sgst_dict['cgst_collection_for_cash'] = main_sale_cash_df['cgst_value'].sum()
            cgst_and_sgst_dict['sgst_collection_for_cash'] = main_sale_cash_df['sgst_value'].sum()
            cgst_and_sgst_dict['total_cash_cost'] = main_sale_cash_df['cash_total_cost'].sum()
        else:
            cgst_and_sgst_dict['cgst_collection_for_cash'] = 0
            cgst_and_sgst_dict['sgst_collection_for_cash'] = 0
            cgst_and_sgst_dict['total_cash_cost'] = 0
        if not main_sale_credit_df.empty:
            cgst_and_sgst_dict['cgst_collection_for_credit'] = main_sale_credit_df['cgst_value'].sum()
            cgst_and_sgst_dict['sgst_collection_for_credit'] = main_sale_credit_df['sgst_value'].sum()
            cgst_and_sgst_dict['total_credit_cost'] = main_sale_credit_df['credit_total_cost'].sum()
        else:
            cgst_and_sgst_dict['cgst_collection_for_credit'] = 0
            cgst_and_sgst_dict['sgst_collection_for_credit'] = 0
            cgst_and_sgst_dict['total_credit_cost'] = 0
        cgst_and_sgst_dict['cgst_account_code'] = '30910'
        cgst_and_sgst_dict['sgst_account_code'] = '30911'
        cgst_and_sgst_dict['misc_account_code'] = '31004'
        if main_sale_group_cash_obj:
            cgst_and_sgst_dict['total_cash_amount'] = main_sale_group_cash_obj.aggregate(Sum('total_cost'))['total_cost__sum']
        else:
            cgst_and_sgst_dict['total_cash_amount'] = 0
        if main_sale_group_credit_obj:
            cgst_and_sgst_dict['total_credit_amount'] = main_sale_group_credit_obj.aggregate(Sum('total_cost'))['total_cost__sum']
        else:
            cgst_and_sgst_dict['total_credit_amount'] = 0
        print( cgst_and_sgst_dict['total_cash_cost'])
        print(cgst_and_sgst_dict['total_credit_cost'])
        print(cgst_and_sgst_dict)
        cash_misc_amount = cgst_and_sgst_dict['total_cash_amount'] - cgst_and_sgst_dict['total_cash_cost']
        credit_misc_amount = cgst_and_sgst_dict['total_credit_amount'] - cgst_and_sgst_dict['total_credit_cost']

        product_group_obj = ByProductGroup.objects.filter()
        product_group_list = list(product_group_obj.values_list('id', 'name', 'account_code'))
        product_group_column = ['group_id', 'name', 'account_code']
        product_group_df = pd.DataFrame(product_group_list, columns=product_group_column)
        product_group_list = product_group_df.to_dict('r')

        df1 = pd.merge(cash_sale_group_wise_df, credit_sale_group_wise_df, left_on='group_id', right_on='group_id', how='left')
        merge_df = pd.merge(df1,product_group_df,left_on='group_id', right_on='group_id', how='left')

        merge_df = merge_df.reindex(columns=['account_code','name',"cost_x",'cost_y'])

        merge_df = merge_df.rename(columns={'account_code':'Code', 'name':'Product',"cost_x":'Cash','cost_y':'Credit'})
    #         new_row = pd.DataFrame({'Code':['30910','30911','31004'],'Product':['CGST Collection','SGST Collection','Misc.Income'],'Cash':[ cgst_and_sgst_dict['cgst_collection_for_cash'],cgst_and_sgst_dict['sgst_collection_for_cash'],cash_misc_amount],'Credit':[cgst_and_sgst_dict['cgst_collection_for_credit'],cgst_and_sgst_dict['sgst_collection_for_credit'],credit_misc_amount]})
    #         merge_df = pd.concat([new_row, merge_df]).reset_index(drop = True)
        merge_df = merge_df.append({'Code':'30910','Product':'CGST Collection','Cash':cgst_and_sgst_dict['cgst_collection_for_cash'],'Credit':cgst_and_sgst_dict['cgst_collection_for_credit']}, ignore_index=True)
        merge_df = merge_df.append({'Code':'30911','Product':'SGST Collection','Cash':cgst_and_sgst_dict['sgst_collection_for_cash'],'Credit':cgst_and_sgst_dict['sgst_collection_for_credit']}, ignore_index=True)
        merge_df = merge_df.append({'Code':'30910','Product':'Misc.Income','Cash':cash_misc_amount,'Credit':credit_misc_amount}, ignore_index=True)
        merge_df = merge_df.append(merge_df[['Cash','Credit']].sum(), ignore_index=True)
        merge_df.iloc[-1, merge_df.columns.get_loc('Product')] = 'TOTAL'


        merge_df.insert(0,"S NO", np.arange(1,len(merge_df)+1))
        merge_df.fillna('')

        credit_business_ids = list(set(main_sale_credit_obj.values_list('by_sale_group__business_id', flat=True)))

        business_obj = Business.objects.filter(id__in=credit_business_ids)
        business_list = list(business_obj.values_list('id', 'businessagentmap__agent__agent_code', 'businessagentmap__agent__first_name'))
        business_column = ['business_id', 'account_code', 'agent_name']
        business_df = pd.DataFrame(business_list, columns=business_column)
        business_df = business_df.fillna(0)
        business_list = business_df.to_dict('r')

        credit_sale_business_wise_df = main_sale_group_credit_df.groupby('business_id').agg({'total_cost': sum})['total_cost']

        df = pd.merge(business_df, credit_sale_business_wise_df, left_on='business_id', right_on='business_id', how='left')
        df = df.reindex(columns=["account_code",'agent_name',"total_cost"])

        df = df.rename(columns={ 'account_code': 'Code','agent_name':'Debitors','total_cost':'Amount'})
        df = df.append(df[['Amount']].sum(), ignore_index=True)
        df.iloc[-1, df.columns.get_loc('Debitors')] = 'TOTAL'
        df.insert(0,"S NO", np.arange(1,len(df)+1))
        df.fillna('')

        result_df = pd.concat([merge_df, df], axis=1)
        # Excel Part
        from_date_in_strip = datetime.datetime.strptime(date_from, "%Y-%m-%d")
        from_date_in_format = datetime.datetime.strftime(from_date_in_strip, '%d-%m-%Y')

        to_date_in_strip = datetime.datetime.strptime(date_to, "%Y-%m-%d")
        to_date_in_format = datetime.datetime.strftime(to_date_in_strip, '%d-%m-%Y')

        excel_file_name = f"{from_date_in_format} - {to_date_in_format} - daily_sales_statement.xlsx"
        excel_file_path = os.path.join('static/media/by_product/daily_sale/',excel_file_name)

        result_df.to_excel(excel_file_path)

#         try:
#             path = excel_file_path
#             os.makedirs(path)
#         except FileExistsError:
#             print('already created')
        document = {}
        document['excel_file_name'] = excel_file_name

        try:
            image_path = excel_file_path
            with open(image_path, 'rb') as image_file:
                encoded_image = b64encode(image_file.read())
                document['excel'] = encoded_image
        except Exception as err:
            print(err)  

        return Response(data=document, status=status.HTTP_200_OK)
#     else:
#         document = {}
#         document['is_data_available'] = False
#         return Response(data=document, status=status.HTTP_200_OK)


In [102]:
result_df

,S NO,Code,Product,Cash,Credit,S NO,Code,Debitors,Amount
0,1,30212,Peda,1813.6000,48854.8000,1.0,8101,RSP UNION PARLOUR,452850.00
1,2,30206,Ghee,131403.0000,346966.2000,2.0,8104,UKKADAM BUS STAND PARLOUR,20845.00
2,3,30216,Badam Power,783.5000,17799.0000,3.0,8115,HI-TECH PARLOUR,43920.00
3,4,30203,F.L.M Bottle,784.8000,17681.4000,4.0,8105,Gandhi Puram Bus Stand Parlour,22415.00
4,5,30214,Mysorepa,923.5200,8000.1600,5.0,8109,V.O.C.PARK PARLOUR,34017.00
5,6,30241,Kajukathri,441.0000,5301.5200,6.0,NaN,TOTAL,574047.00
6,7,30231,Milk Shake,2354.4000,25486.6500,NaN,NaN,NaN,NaN
7,8,30208,Smp Powder,749.2000,NaN,NaN,NaN,NaN,NaN
8,9,30240,Cookies,1555.2000,17139.5000,NaN,NaN,NaN,NaN
9,10,30213,Icecream,4301.9000,24585.2000,NaN,NaN,NaN,NaN


In [24]:
from decimal import Decimal 
for by_product in ByProduct.objects.filter(): 
    in_stock = 0 
    in_receipt = 0 
    if GoodsReceiptRecord.objects.filter(by_product_id=by_product.id).exists(): 
        in_stock = GoodsReceiptRecord.objects.filter(by_product_id=by_product.id).aggregate(Sum('quantity_now'))['quantity_now__sum'] 
        in_receipt = GoodsReceiptRecord.objects.filter(by_product_id=by_product.id).aggregate(Sum('quantity_at_receipt'))['quantity_at_receipt__sum'] 
    orderable_qty = 0 
    total_order = 0 
    if BySale.objects.filter(by_product_id=by_product.id).exists(): 
        total_order = BySale.objects.filter(by_product_id=by_product.id).aggregate(Sum('count'))['count__sum'] 
    if ByProductCurrentAvailablity.objects.filter(by_product_id=by_product.id).exists(): 
        orderable_qty = ByProductCurrentAvailablity.objects.get(by_product_id=by_product.id).quantity_now 
#     difference = orderable_qty - in_stock 
    dif = Decimal(in_receipt) - Decimal(total_order) 
    if difference != 0: 
        print(f"issue at {by_product.short_name} \t {difference} \t {orderable_qty} \t {in_stock} \t {by_product.id} \t {in_receipt} \t {total_order}")

In [23]:
GoodsReceiptRecord.objects.filter(by_product_id=478, quantity_now__gt=0).first().id



2125

In [18]:
GoodsReceiptRecord.objects.filter(by_product_id=470, quantity_now__gt=0).count()

2

In [19]:
GoodsReceiptRecord.objects.filter(by_product_id=470, quantity_now__gt=0).values_list("id")

<QuerySet [(2144,), (2140,)]>

In [1]:
from decimal import Decimal 
for by_product in ByProduct.objects.filter(): 
    in_stock = 0 
    in_receipt = 0 
    if GoodsReceiptRecord.objects.filter(by_product_id=by_product.id).exists(): 
        in_stock = GoodsReceiptRecord.objects.filter(by_product_id=by_product.id).aggregate(Sum('quantity_now'))['quantity_now__sum'] 
        in_receipt = GoodsReceiptRecord.objects.filter(by_product_id=by_product.id).aggregate(Sum('quantity_at_receipt'))['quantity_at_receipt__sum'] 
    orderable_qty = 0 
    total_order = 0 
    if BySale.objects.filter(by_product_id=by_product.id).exists(): 
        total_order = BySale.objects.filter(by_product_id=by_product.id).aggregate(Sum('count'))['count__sum'] 
    if ByProductCurrentAvailablity.objects.filter(by_product_id=by_product.id).exists(): 
        orderable_qty = ByProductCurrentAvailablity.objects.get(by_product_id=by_product.id).quantity_now 
    difference = orderable_qty - in_stock 
    dif = Decimal(in_receipt) - Decimal(total_order) 
    if difference != 0: 
        print(f"issue at {by_product.short_name[0:5]} \t {difference} \t {orderable_qty} \t {in_stock} \t {by_product.id} \t {in_receipt} \t {total_order}")

In [47]:
GoodsReceiptRecord.objects.filter(by_product_id=470, quantity_now__gt=0).count()

2

In [45]:
GoodsReceiptRecord.objects.filter(by_product_id=470, quantity_now__gt=0).first().id



1957

In [49]:
GoodsReceiptRecord.objects.filter(by_product_id=470, quantity_now__gt=0).values_list("id")

<QuerySet [(2188,), (2189,)]>

In [21]:
sale_group_obj = SaleGroup.objects.filter(id=1327868)
                           
            
p= list(sale_group_obj.values_list('total_cost',flat=True))[0]
print(type(p),p)
# print('che',sale_group_obj.total_cost)
# data_dict["cash"][employee_trace_obj.counter.name]["sale_amount"] += float(sale_group_obj['total_cost'])
                

<class 'decimal.Decimal'> 6660.00


In [2]:
def serve_qty_and_short_name(start_date, end_date):
    product_dict = {}
    for product in Product.objects.filter():
        if not product.id in product_dict:
            product_dict[product.id] = {}
        trace_ids = []
        for product_trace in ProductNameAndQuantityTrace.objects.filter(product_id=product.id):
            date_list = pd.date_range(product_trace.start_date, product_trace.end_date)
            if start_date in date_list :
                trace_ids.append(product_trace.id)
            if end_date in date_list:
                trace_ids.append(product_trace.id)
        if len(set(trace_ids)) == 1:
            product_trace_id = trace_ids[0]
            product_trace_obj = ProductNameAndQuantityTrace.objects.get(id=product_trace_id)
            data_dict = {
                'qty': product_trace_obj.quantity,
                'short_name': product_trace_obj.short_name
            }
            product_dict[product.id] = data_dict
        else:
            data_dict = {
                'is_multiple_qty': True,
            }
            return data_dict
        final_dict = {
            'is_multiple_qty' : False,
            'product_dict': product_dict
        }
        return final_dict


In [18]:
# def serve_govt_jst_bill_summary(request):
# print(request.data)
# selected_business_type_ids = request.data['selected_business_type_id']
# start_date = request.data['start_date']
# end_date = request.data['end_date']
import pandas as pd
import datetime
# import timezone
selected_business_type_ids = [1]
start_date = '2022-10-01'
end_date = '2022-10-01'
business_codes = list(Business.objects.filter(business_type_id__in=selected_business_type_ids).values_list('code', flat=True))

business_wise_bill = BusinessWiseBillNumber.objects.filter(business__code__in=business_codes, from_date=start_date, to_date=end_date)
business_wise_bill_list = list(business_wise_bill.values_list('business_id', 'bill_number'))
business_wise_bill_columns = ['business_id', 'bill_number']
business_wise_bill_df = pd.DataFrame(business_wise_bill_list, columns=business_wise_bill_columns)

# date_in_format = datetime.strptime(str(end_date), '%Y-%m-%d')
# date = datetime.strftime(date_in_format, '%d-%b-%Y')
date='2022-10-01'
product_quantity_check_dict = serve_qty_and_short_name(start_date, end_date)
print(product_quantity_check_dict)
sale_obj = Sale.objects.filter(sale_group__business__code__in=business_codes, sale_group__date__gte=start_date, sale_group__date__lte=end_date, product__group_id=2).exclude(product_id=28)
sale_list = list(sale_obj.values_list('id', 'sale_group__business__user_profile_id','sale_group__business_id', 'sale_group__business__code', 'product_id', 'count', 'product__cgst_percent', 'product__sgst_percent', 'cost', 'product__hsn_code', 'sale_group__business__businessagentmap__agent__first_name', 'sale_group__business__businessagentmap__agent__agent_profile__gst_number'))
sale_column = ['id', 'user_profile_id', 'business_id', 'business_code', 'product_id', 'count', 'cgst_percent', 'sgst_percent' ,'cost', 'hsn_code', 'agent_name', 'gst_number']
sale_df = pd.DataFrame(sale_list, columns=sale_column)

sale_df = sale_df.groupby(['business_id', 'product_id']).agg({'count': sum, 'user_profile_id': 'first', 'business_code': 'first', 'cgst_percent': 'first', 'sgst_percent': 'first', 'cost': sum, 'hsn_code': 'first', 'agent_name': 'first', 'gst_number': 'first'}).reset_index()
short_name_and_qty = product_quantity_check_dict['product_dict']
print(short_name_and_qty)
if sale_df.empty:
    document = {}
    document['is_data_available'] = False
    return Response(data=document, status=status.HTTP_200_OK)
sale_df['cost'] = sale_df['cost'].astype(float)
sale_df['mrp'] = sale_df['cost'] / sale_df['count']
sale_df['product_quantity'] = sale_df.apply(lambda x: short_name_and_qty[x['product_id']]['qty'], axis=1)
sale_df['product_short_name'] = sale_df.apply(lambda x: short_name_and_qty[x['product_id']]['short_name'], axis=1)
sale_df['cost_by_base_price'] = sale_df.apply(lambda x: find_base_price_from_total_cost(x['cost'], x['cgst_percent'] + x['sgst_percent']), axis=1)
sale_df['cost_by_base_price'] = sale_df['cost_by_base_price'].astype(float)
sale_df['base_price'] = sale_df['cost_by_base_price'] / sale_df['count']
sale_df['product_quantity'] = sale_df['product_quantity'].astype(float)
sale_df['total_gst_for_base_price'] = sale_df['cost'] - sale_df['cost_by_base_price']
sale_df['qty_in_whole'] = (sale_df['count'] * sale_df['product_quantity']) / 1000
sale_df['total_gst'] = sale_df['cgst_percent'] + sale_df['sgst_percent']
sale_df['total_cost'] = (sale_df['cost_by_base_price'] + sale_df['total_gst_for_base_price'])

sale_df = sale_df.merge(business_wise_bill_df, how="left", on="business_id").fillna("")
sale_df = sale_df[sale_df['bill_number'] != ""]

# sheet 1
grouped_sale_df = sale_df.groupby(['business_id', 'total_gst']).agg({'agent_name': 'first','gst_number': 'first','bill_number': 'first', 'total_cost': 'sum','cost_by_base_price': sum , 'total_gst_for_base_price': sum}).reset_index()
grouped_sale_df['total_cost_rounded'] = grouped_sale_df['total_cost'].apply(np.ceil)
grouped_sale_df['cgst_value'] = grouped_sale_df['total_gst_for_base_price'] / 2
grouped_sale_df['sgst_value'] = grouped_sale_df['total_gst_for_base_price'] / 2
grouped_sale_df['bill_date'] = date

final_df = grouped_sale_df.rename(columns = {'gst_number': 'GSTIN_UINOFRECIPIENT', 'agent_name': 'NAME', 'bill_number': 'INVOICENO', 'bill_date' : 'INVOICEDATE', 'total_cost_rounded': 'INVOICEVALUE', 'total_gst': 'RATE', 'cost_by_base_price': 'TAXABLEVALUE', 'cgst_value': 'CGST', 'sgst_value': 'SGST'})
final_df['PLACEOFSUPPLY'] = ''
final_df['REVERSECHARGE'] = ''
final_df['INVOICETYPE'] = ''
final_df['IGST'] = 0
final_df = final_df[['GSTIN_UINOFRECIPIENT', 'NAME', 'INVOICENO', 'INVOICEDATE', 'INVOICEVALUE', 'PLACEOFSUPPLY', 'REVERSECHARGE', 'INVOICETYPE', 'RATE', 'TAXABLEVALUE', 'CGST', 'SGST', 'IGST']]

# sheet 2
product_wise_df = sale_df
product_wise_df['bill_date'] = date
product_wise_df['cgst_value'] = product_wise_df['total_gst_for_base_price'] / 2
product_wise_df['sgst_value'] = product_wise_df['total_gst_for_base_price'] / 2
product_wise_df['ISGT'] = 0.000
product_wise_df = product_wise_df.rename(columns={'agent_name': 'AGENT', 'gst_number': 'BTAXNO', 'product_short_name': 'PRODUCT', 'bill_number': 'INVNO', 'bill_date': 'INVDATE', 'sgst_value': 'SGST','cgst_value': 'CGST', 'total_cost': 'NETAMOUNT', 'hsn_code': 'HSNCODE', 'total_gst': 'GST', 'mrp': 'RATE'})
final_df_2 = product_wise_df[['AGENT', 'BTAXNO', 'PRODUCT', 'INVNO', 'INVDATE', 'RATE', 'GST', 'SGST', 'CGST', 'ISGT','NETAMOUNT', 'HSNCODE']]

file_name = f"static/media/monthly_report/{start_date} - {end_date} - GVT Fermented GST Bill Summary.xlsx"
writer = pd.ExcelWriter(file_name, engine="xlsxwriter")
sheet_name1 = "sheet1"
sheet_name2 = "sheet2"
final_df.to_excel(writer, sheet_name=sheet_name1, index=False)
final_df_2.to_excel(writer, sheet_name=sheet_name2, index=False)
writer.save()
document = {}
document['is_data_available'] = True
document['file_name'] = f"{start_date} - {end_date} - GVT Fermented GST Bill Summary.xlsx"
try:
    image_path = file_name
    with open(image_path, 'rb') as image_file:
        encoded_image = b64encode(image_file.read())
        document['excel'] = encoded_image
except Exception as err:
    print(err)
    document = {}
return Response(data=document, status=status.HTTP_200_OK)


{'is_multiple_qty': False, 'product_dict': {22: {'qty': Decimal('1000.000'), 'short_name': 'TNcan'}}}
{22: {'qty': Decimal('1000.000'), 'short_name': 'TNcan'}}


KeyError: 25

In [11]:
short_name_and_qty


NameError: name 'short_name_and_qty' is not defined

##  Price change for FCM 500 and FCM 1000

In [44]:
product_price_for_agent = {
    4: 30,
    6: 60
}
# product_price_for_icustomer = {
#      4: 30,
# }
# product_price_for_society = {
    
#     4: 28.77,
#     6: 57.54
# }
# product_price_for_society = {
    
#     4: 28.77,
#     6: 57.54
# }

In [45]:
product_price_for_agent

{4: 30, 6: 60}

In [29]:
# product price update on mrp

for product in Product.objects.filter(group_id=1):
    if product.id in product_price_for_agent:
        print(f"Old Price {product.mrp} - updated price {product_price_for_agent[product.id]}")
        product.mrp = product_price_for_agent[product.id]
        product.save()

Old Price 24.00 - updated price 30
Old Price 60.00 - updated price 60


In [14]:
for business in BusinessWiseProductDiscount.objects.filter(business_id__in=m_27_10, product__group_id=1,product_id=6):
        business.discounted_price = 
        print(business.discounted_price)
        business.save()

27.1
27.1


In [5]:
for business in BusinessWiseProductDiscount.objects.filter(business_id__in=m_NA, product__group_id=1,product_id__in=[6,4]):
#         business.discounted_price = 28.77
#         print(business.discounted_price)
    print("r")
    business.delete()

r
r
r
r
r


In [ ]:
for business in BusinessWiseProductDiscount.objects.filter(business_id__in=m_27_10, product__group_id=1,product_id=6):
        business.discounted_price = 
        print(business.discounted_price)
        business.save()

In [10]:
for businessdiscount in BusinessWiseProductDiscountTrace.objects.filter(business_wise_discount__business_id__in=m_28_77, business_wise_discount__product__group_id=1,business_wise_discount__product_id=6):
    businessdiscount.discounted_price=57.54
    businessdiscount.mrp=60
    businessdiscount.save()

In [9]:
m_28_77 = [1576,1735,2399,2525,3542,3543,3544,3545,3546,3547,3548,3549,3550,3551,3552,3553,3554,3555,3556,3557,3558,3559,3560,3561,3562,3563,3564,3565,3566,3567,3568,3569]
len(m_28_77)

32

In [ ]:
m=[]

In [39]:
business_obj=Business.objects.filter().values_list('id','name')
colum=['id','name']
df= pd.DataFrame(business_obj,columns=colum)
df.to_excel("naveen.xlsx")

In [40]:
df

,id,name
0,1291,84 - Booth
1,1313,260 - Booth
2,1324,354 - Booth
3,1326,392 - Booth
4,1327,411 - Booth
...,...,...
2226,3697,70401
2227,3698,3095 - MANICKAVASAGAM.K.R
2228,3699,95001
2229,3700,3019 - GAYATHRI.R


In [7]:
m_30=[1439,
1440,1441
,1501
,1513
,1514
,1521
,1522
,1525
,1528
,1535
,1538
,1539
,1540
,1542
,1543
,1545
,1549,
1552,
1553,
1554,
1557,
1558,
1559,
1564,
1566,
1571,
1572,
1573,
1575,
1744,
1745,
1746,
1747,
1748,
1749,
1750,
1751,
1752,
1754,
1755,
1758,
1759,
1762,
1763,
1765,
1767,
1768,
1769,
1771,
1772,
1773,
1774,
1861,
1947,
2011,
2012,
2013,
2014,
2015,
2016,
2018,
2019,
2020,
2021,
2026,
2027,
2029,
2031,
2034,
2036,
2039,
2040,
2041,
2042,
2044,
2046,
2047,
2050,
2052,
2053,
2054,
2055,
2056,
2058,
2130,
2133,
2134,
2141,
2142,
2143,
2144,
2145,
2146,
2147,
2148,
2149,
2150,
2151,
2152,
2154,
2155,
2156,
2335,
2399,
2400,
2404,
2405,
2409,
2410,
2412,
2413,
2414,
2419,
2420,
2421,
2422,
2424,
2425,
2426,
2427,
2430,
2431,
2432,
2526,
2529,
2530,
2531,
2532,
2533,
2534,
2535,
2542,
2543,
2548,
2549,
2550,
2551,
2552,
2553,
2554,
2555,
2746,
2747,
2779,
3216,
3267,
3293,
3331,
3366,
3435,
3583,
]

In [43]:
l

[1439,
 1440,
 1441,
 1501,
 1513,
 1514,
 1521,
 1522,
 1525,
 1528,
 1535,
 1538,
 1539,
 1540,
 1542,
 1543,
 1545,
 1549,
 1552,
 1553,
 1554,
 1557,
 1558,
 1559,
 1564,
 1566,
 1571,
 1572,
 1573,
 1575,
 1744,
 1745,
 1746,
 1747,
 1748,
 1749,
 1750,
 1751,
 1752,
 1754,
 1755,
 1758,
 1759,
 1762,
 1763,
 1765,
 1767,
 1768,
 1769,
 1771,
 1772,
 1773,
 1774,
 1861,
 1947,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2018,
 2019,
 2020,
 2021,
 2026,
 2027,
 2029,
 2031,
 2034,
 2036,
 2039,
 2040,
 2041,
 2042,
 2044,
 2046,
 2047,
 2050,
 2052,
 2053,
 2054,
 2055,
 2056,
 2058,
 2130,
 2133,
 2134,
 2141,
 2142,
 2143,
 2144,
 2145,
 2146,
 2147,
 2148,
 2149,
 2150,
 2151,
 2152,
 2154,
 2155,
 2156,
 2335,
 2399,
 2400,
 2404,
 2405,
 2409,
 2410,
 2412,
 2413,
 2414,
 2419,
 2420,
 2421,
 2422,
 2424,
 2425,
 2426,
 2427,
 2430,
 2431,
 2432,
 2526,
 2529,
 2530,
 2531,
 2532,
 2533,
 2534,
 2535,
 2542,
 2543,
 2548,
 2549,
 2550,
 2551,
 2552,
 2553,
 2554,
 2555,
 2746,

In [10]:
m_28_77=[1442,1512,1516,1518,1519,1520,1523,1524,1526,1527,1529,1530,1531,1532,1533,1534,1536,1537,1541,1544,1546,1547,1548,1550,1551,1555,1556,1560,1561,1562,1565,1567,1568,1569,1570,1574,1576,1735,1753,1756,1757,1760,1761,1764,1766,1770,2017,2023,2024,2025,2028,2030,2032,2033,2035,2037,2038,2043,2045,2048,2049,2051,2384,2389,2401,2402,2403,2406,2407,2408,2411,2415,2416,2417,2418,2423,2428,2429,2525,2527,2528,2544,2545,2546,2547,2568,2748,2749,2750,2751,2752,2761,2773,2774,2775,2780,2781,2790,2794,2797,2799,2815,2816,2822,2825,3228,3268,3433,3542,3543,3544,3545,3546,3547,3548,3549,3550,3551,3552,3553,3554,3555,3556,3557,3558,3559,3560,3561,3562,3563,3564,3565,3566,3567,3568,3569
]

In [13]:
m_27_10=[2536,2874]

In [1]:
m_NA=[1515,1517,1563,2022,2881]

In [19]:
# def serve_govt_jst_bill_summary(request):
# print(request.data)
# selected_business_type_ids = request.data['selected_business_type_id']
# start_date = request.data['start_date']
# end_date = request.data['end_date']
import pandas as pd
import datetime
# import timezone
selected_business_type_ids = [1]
start_date = '2022-10-01'
end_date = '2022-10-01'
business_codes = list(Business.objects.filter(business_type_id__in=selected_business_type_ids).values_list('code', flat=True))

business_wise_bill = BusinessWiseBillNumber.objects.filter(business__code__in=business_codes, from_date=start_date, to_date=end_date)
business_wise_bill_list = list(business_wise_bill.values_list('business_id', 'bill_number'))
business_wise_bill_columns = ['business_id', 'bill_number']
business_wise_bill_df = pd.DataFrame(business_wise_bill_list, columns=business_wise_bill_columns)

# date_in_format = datetime.strptime(str(end_date), '%Y-%m-%d')
# date = datetime.strftime(date_in_format, '%d-%b-%Y')
date='2022-10-01'
product_quantity_check_dict = serve_qty_and_short_name(start_date, end_date)
print(product_quantity_check_dict)
sale_obj = Sale.objects.filter(sale_group__business__code__in=business_codes, sale_group__date__gte=start_date, sale_group__date__lte=end_date, product__group_id=2).exclude(product_id=28)
sale_list = list(sale_obj.values_list('id', 'sale_group__business__user_profile_id','sale_group__business_id', 'sale_group__business__code', 'product_id', 'count', 'product__cgst_percent', 'product__sgst_percent', 'cost', 'product__hsn_code', 'sale_group__business__businessagentmap__agent__first_name', 'sale_group__business__businessagentmap__agent__agent_profile__gst_number'))
sale_column = ['id', 'user_profile_id', 'business_id', 'business_code', 'product_id', 'count', 'cgst_percent', 'sgst_percent' ,'cost', 'hsn_code', 'agent_name', 'gst_number']
sale_df = pd.DataFrame(sale_list, columns=sale_column)

sale_df = sale_df.groupby(['business_id', 'product_id']).agg({'count': sum, 'user_profile_id': 'first', 'business_code': 'first', 'cgst_percent': 'first', 'sgst_percent': 'first', 'cost': sum, 'hsn_code': 'first', 'agent_name': 'first', 'gst_number': 'first'}).reset_index()
short_name_and_qty = product_quantity_check_dict['product_dict']
print(short_name_and_qty)
if sale_df.empty:
    document = {}
    document['is_data_available'] = False
#     return Response(data=document, status=status.HTTP_200_OK)
sale_df['cost'] = sale_df['cost'].astype(float)
sale_df['mrp'] = sale_df['cost'] / sale_df['count']
sale_df['product_quantity'] = sale_df.apply(lambda x: short_name_and_qty[x['product_id']]['qty'], axis=1)
sale_df['product_short_name'] = sale_df.apply(lambda x: short_name_and_qty[x['product_id']]['short_name'], axis=1)
sale_df['cost_by_base_price'] = sale_df.apply(lambda x: find_base_price_from_total_cost(x['cost'], x['cgst_percent'] + x['sgst_percent']), axis=1)
sale_df['cost_by_base_price'] = sale_df['cost_by_base_price'].astype(float)
sale_df['base_price'] = sale_df['cost_by_base_price'] / sale_df['count']
sale_df['product_quantity'] = sale_df['product_quantity'].astype(float)
sale_df['total_gst_for_base_price'] = sale_df['cost'] - sale_df['cost_by_base_price']
sale_df['qty_in_whole'] = (sale_df['count'] * sale_df['product_quantity']) / 1000
sale_df['total_gst'] = sale_df['cgst_percent'] + sale_df['sgst_percent']
sale_df['total_cost'] = (sale_df['cost_by_base_price'] + sale_df['total_gst_for_base_price'])

sale_df = sale_df.merge(business_wise_bill_df, how="left", on="business_id").fillna("")
sale_df = sale_df[sale_df['bill_number'] != ""]

# sheet 1
grouped_sale_df = sale_df.groupby(['business_id', 'total_gst']).agg({'agent_name': 'first','gst_number': 'first','bill_number': 'first', 'total_cost': 'sum','cost_by_base_price': sum , 'total_gst_for_base_price': sum}).reset_index()
grouped_sale_df['total_cost_rounded'] = grouped_sale_df['total_cost'].apply(np.ceil)
grouped_sale_df['cgst_value'] = grouped_sale_df['total_gst_for_base_price'] / 2
grouped_sale_df['sgst_value'] = grouped_sale_df['total_gst_for_base_price'] / 2
grouped_sale_df['bill_date'] = date

final_df = grouped_sale_df.rename(columns = {'gst_number': 'GSTIN_UINOFRECIPIENT', 'agent_name': 'NAME', 'bill_number': 'INVOICENO', 'bill_date' : 'INVOICEDATE', 'total_cost_rounded': 'INVOICEVALUE', 'total_gst': 'RATE', 'cost_by_base_price': 'TAXABLEVALUE', 'cgst_value': 'CGST', 'sgst_value': 'SGST'})
final_df['PLACEOFSUPPLY'] = ''
final_df['REVERSECHARGE'] = ''
final_df['INVOICETYPE'] = ''
final_df['IGST'] = 0
final_df = final_df[['GSTIN_UINOFRECIPIENT', 'NAME', 'INVOICENO', 'INVOICEDATE', 'INVOICEVALUE', 'PLACEOFSUPPLY', 'REVERSECHARGE', 'INVOICETYPE', 'RATE', 'TAXABLEVALUE', 'CGST', 'SGST', 'IGST']]

# sheet 2
product_wise_df = sale_df
product_wise_df['bill_date'] = date
product_wise_df['cgst_value'] = product_wise_df['total_gst_for_base_price'] / 2
product_wise_df['sgst_value'] = product_wise_df['total_gst_for_base_price'] / 2
product_wise_df['ISGT'] = 0.000
product_wise_df = product_wise_df.rename(columns={'agent_name': 'AGENT', 'gst_number': 'BTAXNO', 'product_short_name': 'PRODUCT', 'bill_number': 'INVNO', 'bill_date': 'INVDATE', 'sgst_value': 'SGST','cgst_value': 'CGST', 'total_cost': 'NETAMOUNT', 'hsn_code': 'HSNCODE', 'total_gst': 'GST', 'mrp': 'RATE'})
final_df_2 = product_wise_df[['AGENT', 'BTAXNO', 'PRODUCT', 'INVNO', 'INVDATE', 'RATE', 'GST', 'SGST', 'CGST', 'ISGT','NETAMOUNT', 'HSNCODE']]

file_name = f"static/media/monthly_report/{start_date} - {end_date} - GVT Fermented GST Bill Summary.xlsx"
writer = pd.ExcelWriter(file_name, engine="xlsxwriter")
sheet_name1 = "sheet1"
sheet_name2 = "sheet2"
final_df.to_excel(writer, sheet_name=sheet_name1, index=False)
final_df_2.to_excel(writer, sheet_name=sheet_name2, index=False)
writer.save()
document = {}
document['is_data_available'] = True
document['file_name'] = f"{start_date} - {end_date} - GVT Fermented GST Bill Summary.xlsx"
# try:
#     image_path = file_name
#     with open(image_path, 'rb') as image_file:
#         encoded_image = b64encode(image_file.read())
#         document['excel'] = encoded_image
# except Exception as err:
#     print(err)
#     document = {}
# return Response(data=document, status=status.HTTP_200_OK)


{'is_multiple_qty': False, 'product_dict': {22: {'qty': Decimal('1000.000'), 'short_name': 'TNcan'}}}
{22: {'qty': Decimal('1000.000'), 'short_name': 'TNcan'}}


KeyError: 25

ValueError: No axis named cost for object type DataFrame

In [17]:
sale_df

,business_id,product_id,count,user_profile_id,business_code,cgst_percent,sgst_percent,cost,hsn_code,agent_name,gst_number,mrp
0,1289,25,14.0,1300,14,2.50,2.50,446.74,0403,RAGHAVAN.P,None,31.91
1,1292,7,20.0,1303,90,2.50,2.50,166.00,0403,NAGARAJ .M - [1525],None,8.30
2,1292,25,20.0,1303,90,2.50,2.50,638.20,0403,NAGARAJ .M - [1525],None,31.91
3,1295,7,36.0,1306,105,2.50,2.50,298.80,0403,PALANISAMY .M,None,8.30
4,1295,25,60.0,1306,105,2.50,2.50,1914.60,0403,PALANISAMY .M,None,31.91
...,...,...,...,...,...,...,...,...,...,...,...,...
323,3436,7,20.0,24610,3006,2.50,2.50,166.00,0403,ASHOK KUMAR.R.,None,8.30
324,3531,7,10.0,24855,3018,2.50,2.50,83.00,0403,NANDHAGOPAL.R,None,8.30
325,3578,7,20.0,25020,3078,2.50,2.50,166.00,0403,SUGAPRIYA.V,None,8.30
326,3581,7,25.0,25055,3081,2.50,2.50,207.50,0403,SRIKANTH.R,None,8.30


In [18]:
grouped_sale_df

NameError: name 'grouped_sale_df' is not defined

In [24]:
# def serve_govt_jst_bill_summary(request):
# print(request.data)
# selected_business_type_ids = request.data['selected_business_type_id']
# start_date = request.data['start_date']
# end_date = request.data['end_date']

selected_business_type_ids = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
start_date = "2022-09-01"
end_date = "2022-09-30"

business_codes = list(Business.objects.filter(business_type_id__in=selected_business_type_ids).values_list('code', flat=True))

business_wise_bill = BusinessWiseBillNumber.objects.filter(business__code__in=business_codes, from_date=start_date, to_date=end_date)
business_wise_bill_list = list(business_wise_bill.values_list('business_id', 'bill_number'))
business_wise_bill_columns = ['business_id', 'bill_number']
business_wise_bill_df = pd.DataFrame(business_wise_bill_list, columns=business_wise_bill_columns)

# date_in_format = datetime.strptime(str(end_date), '%Y-%m-%d')
# date = datetime.strftime(date_in_format, '%d-%b-%Y')
product_quantity_check_dict = serve_qty_and_short_name(start_date, end_date)
sale_obj = Sale.objects.filter(sale_group__business__code__in=business_codes,sale_ ,sale_group__date__gte=start_date, sale_group__date__lte=end_date, product__group_id=2).exclude(product_id=28)
sale_list = list(sale_obj.values_list('id', 'sale_group__business__user_profile_id','sale_group__business_id', 'sale_group__business__code', 'product_id', 'count', 'product__cgst_percent', 'product__sgst_percent', 'cost', 'product__hsn_code', 'sale_group__business__businessagentmap__agent__first_name', 'sale_group__business__businessagentmap__agent__agent_profile__gst_number'))
sale_column = ['id', 'user_profile_id', 'business_id', 'business_code', 'product_id', 'count', 'cgst_percent', 'sgst_percent' ,'cost', 'hsn_code', 'agent_name', 'gst_number']
sale_df = pd.DataFrame(sale_list, columns=sale_column)

sale_df = sale_df.groupby(['business_id', 'product_id']).agg({'count': sum, 'user_profile_id': 'first', 'business_code': 'first', 'cgst_percent': 'first', 'sgst_percent': 'first', 'cost': sum, 'hsn_code': 'first', 'agent_name': 'first', 'gst_number': 'first'}).reset_index()
short_name_and_qty = product_quantity_check_dict['product_dict']

if sale_df.empty:
    document = {}
    document['is_data_available'] = False
#     return Response(data=document, status=status.HTTP_200_OK)
sale_df['cost'] = sale_df['cost'].astype(float)
sale_df['mrp'] = sale_df['cost'] / sale_df['count']
sale_df['product_quantity'] = sale_df.apply(lambda x: short_name_and_qty[x['product_id']]['qty'], axis=1)
sale_df['product_short_name'] = sale_df.apply(lambda x: short_name_and_qty[x['product_id']]['short_name'], axis=1)
sale_df['cost_by_base_price'] = sale_df.apply(lambda x: find_base_price_from_total_cost(x['cost'], x['cgst_percent'] + x['sgst_percent']), axis=1)
sale_df['cost_by_base_price'] = sale_df['cost_by_base_price'].astype(float)
sale_df['base_price'] = sale_df['cost_by_base_price'] / sale_df['count']
sale_df['product_quantity'] = sale_df['product_quantity'].astype(float)
sale_df['total_gst_for_base_price'] = sale_df['cost'] - sale_df['cost_by_base_price']
sale_df['qty_in_whole'] = (sale_df['count'] * sale_df['product_quantity']) / 1000
sale_df['total_gst'] = sale_df['cgst_percent'] + sale_df['sgst_percent']
sale_df['total_cost'] = (sale_df['cost_by_base_price'] + sale_df['total_gst_for_base_price'])

sale_df = sale_df.merge(business_wise_bill_df, how="left", on="business_id")
sale_df = sale_df[sale_df['bill_number'] != ""]

# sheet 1
grouped_sale_df = sale_df.groupby(['business_id', 'total_gst']).agg({'agent_name': 'first','gst_number': 'first','bill_number': 'first', 'total_cost': 'sum','cost_by_base_price': sum , 'total_gst_for_base_price': sum}).reset_index()
grouped_sale_df['total_cost_rounded'] = grouped_sale_df['total_cost'].apply(np.ceil)
grouped_sale_df['cgst_value'] = grouped_sale_df['total_gst_for_base_price'] / 2
grouped_sale_df['sgst_value'] = grouped_sale_df['total_gst_for_base_price'] / 2
grouped_sale_df['bill_date'] = "2022-09-01"

final_df = grouped_sale_df.rename(columns = {'gst_number': 'GSTIN_UINOFRECIPIENT', 'agent_name': 'NAME', 'bill_number': 'INVOICENO', 'bill_date' : 'INVOICEDATE', 'total_cost_rounded': 'INVOICEVALUE', 'total_gst': 'RATE', 'cost_by_base_price': 'TAXABLEVALUE', 'cgst_value': 'CGST', 'sgst_value': 'SGST'})
final_df['PLACEOFSUPPLY'] = ''
final_df['REVERSECHARGE'] = ''
final_df['INVOICETYPE'] = ''
final_df['IGST'] = 0
final_df = final_df[['GSTIN_UINOFRECIPIENT', 'NAME', 'INVOICENO', 'INVOICEDATE', 'INVOICEVALUE', 'PLACEOFSUPPLY', 'REVERSECHARGE', 'INVOICETYPE', 'RATE', 'TAXABLEVALUE', 'CGST', 'SGST', 'IGST']]

# sheet 2
product_wise_df = sale_df
product_wise_df['bill_date'] = date
product_wise_df['total_cost_rounded'] = product_wise_df['total_cost'].apply(np.ceil)
product_wise_df['cgst_value'] = product_wise_df['total_gst_for_base_price'] / 2
product_wise_df['sgst_value'] = product_wise_df['total_gst_for_base_price'] / 2
product_wise_df['ISGT'] = 0.000
product_wise_df = product_wise_df.rename(columns={'agent_name': 'AGENT', 'gst_number': 'BTAXNO', 'product_short_name': 'PRODUCT', 'bill_number': 'INVNO', 'bill_date': 'INVDATE', 'sgst_value': 'SGST','cgst_value': 'CGST', 'total_cost_rounded': 'NETAMOUNT', 'hsn_code': 'HSNCODE', 'total_gst': 'GST', 'mrp': 'RATE'})
final_df_2 = product_wise_df[['AGENT', 'BTAXNO', 'PRODUCT', 'INVNO', 'INVDATE', 'RATE', 'GST', 'SGST', 'CGST', 'ISGT','NETAMOUNT', 'HSNCODE']]

file_name = f"static/media/monthly_report/{start_date} - {end_date} - GVT Fermented GST Bill Summary.xlsx"
writer = pd.ExcelWriter(file_name, engine="xlsxwriter")
sheet_name1 = "sheet1"
sheet_name2 = "sheet2"
final_df.to_excel(writer, sheet_name=sheet_name1, index=False)
final_df_2.to_excel(writer, sheet_name=sheet_name2, index=False)
writer.save()
document = {}
document['is_data_available'] = True
document['file_name'] = f"{start_date} - {end_date} - GVT Fermented GST Bill Summary.xlsx"
# try:
#     image_path = file_name
#     with open(image_path, 'rb') as image_file:
#         encoded_image = b64encode(image_file.read())
#         document['excel'] = encoded_image
# except Exception as err:
#     print(err)
#     document = {}
# return Response(data=document, status=status.HTTP_200_OK)



KeyError: 25

In [11]:
def serve_qty_and_short_name(start_date, end_date):
    product_dict = {}
    for product in Product.objects.filter():
        if not product.id in product_dict:
            product_dict[product.id] = {}
        trace_ids = []
        for product_trace in ProductNameAndQuantityTrace.objects.filter(product_id=product.id):
            date_list = pd.date_range(product_trace.start_date, product_trace.end_date)
            if start_date in date_list :
                trace_ids.append(product_trace.id)
            if end_date in date_list:
                trace_ids.append(product_trace.id)
        if len(set(trace_ids)) == 1:
            product_trace_id = trace_ids[0]
            product_trace_obj = ProductNameAndQuantityTrace.objects.get(id=product_trace_id)
            data_dict = {
                'qty': product_trace_obj.quantity,
                'short_name': product_trace_obj.short_name
            }
            product_dict[product.id] = data_dict
        else:
            data_dict = {
                'is_multiple_qty': True,
            }
            return data_dict
        final_dict = {
            'is_multiple_qty' : False,
            'product_dict': product_dict
        }
        return final_dict


In [34]:
sale_df.to_excel('name.xlsx')
sale_df

,business_id,product_id,count,user_profile_id,business_code,cgst_percent,sgst_percent,cost,hsn_code,agent_name,gst_number,mrp
0,1289,25,412.0,1300,14,2.50,2.50,13146.92,0403,RAGHAVAN.P,None,31.910000
1,1292,7,1190.0,1303,90,2.50,2.50,12370.20,0403,NAGARAJ .M - [1525],None,10.395126
2,1292,25,410.0,1303,90,2.50,2.50,13083.10,0403,NAGARAJ .M - [1525],None,31.910000
3,1293,7,64.0,1304,91,2.50,2.50,715.48,0403,SENTHILKUMAR.M,None,11.179375
4,1295,7,496.0,1306,105,2.50,2.50,4929.80,0403,PALANISAMY .M,None,9.939113
...,...,...,...,...,...,...,...,...,...,...,...,...
666,3541,7,420.0,24914,3016,2.50,2.50,4082.20,0403,UMAPATHY.R,None,9.719524
667,3541,8,54.0,24914,3016,2.50,2.50,392.58,0403,UMAPATHY.R,None,7.270000
668,3541,25,147.0,24914,3016,2.50,2.50,4690.77,0403,UMAPATHY.R,None,31.910000
669,3578,7,80.0,25020,3078,2.50,2.50,664.00,0403,SUGAPRIYA.V,None,8.300000


In [35]:
grouped_sale_df

NameError: name 'grouped_sale_df' is not defined

In [6]:
sale_group_obj=SaleGroup.objects.filter(business_type_id=1,business_id=1289)

In [7]:
sale_group_list = list(sale_group_obj)

<QuerySet [<SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, <SaleGroup: SaleGroup object>, '...(remaining elements truncated)...']>

In [20]:
# def serve_fermented_products_gst_bill(request):
# print(request.data)
# selected_business_type_ids = request.data['selected_business_type_id']
# start_date = request.data['start_date']
# end_date = request.data['end_date']
import pandas as pd
import datetime


selected_business_type_ids = [1]
start_date = "2022-10-01"
end_date = "2022-10-31"

if SaleGroup.objects.filter(date__range=[start_date, end_date], business__business_type_id__in=selected_business_type_ids, sale__product__group_id=2).exists():
    sale_obj = SaleGroup.objects.filter(date__range=[start_date, end_date], business__business_type_id__in=selected_business_type_ids, sale__product__group_id=2)
    sale_values = list(sale_obj.values_list('business__businessagentmap__agent__first_name', 'business__businessagentmap__agent__agent_profile__gst_number', 'date', 'total_cost', 'sale__product__gst_percent', 'sale__cost','id'))
    sale_column = ['agent_name', 'gst_number', 'bill_date', 'total_cost', 'gst_percent', 'cost', 'bill_id']

    sale_df = pd.DataFrame(sale_values, columns=sale_column)
    sale_df['bill_date'] = pd.to_datetime(sale_df['bill_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime("%Y-%m-%d")
#     sale_df['bill_date'] = "2022-10-01"
    default_value=1
    sale_df['gst']=default_value+(sale_df['gst_percent'].astype('float')/100)
    sale_df['cost']=sale_df['total_cost'].astype('float')/sale_df['gst'].astype('float')
    sale_df['total_gst']=sale_df['cost'].astype('float')*(sale_df['gst_percent'].astype('float')/100)
    sale_df['cgst']=(sale_df['total_gst'].astype('float'))/2
    sale_df['sgst']=(sale_df['total_gst'].astype('float'))/2

    sale_df = sale_df.fillna('')
    
    sale_df = sale_df.groupby(['bill_id', 'gst']).agg({'agent_name': 'first','gst_number': 'first','bill_date': 'first', 'total_cost': 'first','gst_percent':sum,'cost': sum ,'cgst':sum,'sgst':sum}).reset_index()
    final_df = sale_df.rename(columns = {'gst_number': 'GSTIN_UINOFRECIPIENT', 'agent_name': 'NAME','bill_date' : 'INVOICEDATE', 'total_cost': 'INVOICEVALUE', 'gst_percent': 'RATE', 'cost': 'TAXABLEVALUE','cgst':'CGST','sgst':'SGST'})
    final_df['PLACEOFSUPPLY'] = ''
    final_df['REVERSECHARGE'] = ''
    final_df['INVOICETYPE'] = ''
    final_df['IGST'] = 0
    final_df['INVOICENO'] = ""
    final_df = final_df.drop(columns=['bill_id'])
    final_df = final_df[['GSTIN_UINOFRECIPIENT', 'NAME', 'INVOICENO', 'INVOICEDATE', 'INVOICEVALUE', 'PLACEOFSUPPLY', 'REVERSECHARGE', 'INVOICETYPE', 'RATE', 'TAXABLEVALUE','CGST','SGST','IGST']]
    final_df.round(2)
    sale_group_gatepass_obj = SaleGroup.objects.filter(date__range=[start_date, end_date], business__business_type_id__in=selected_business_type_ids, sale__product__group_id=2).values_list('business__businessagentmap__agent__first_name', 'business__businessagentmap__agent__agent_profile__gst_number', 'sale__product__short_name','date', 'sale__product__gst_percent','sale__cost', 'sale__count', 'sale__product__hsn_code')
    sale_group_gatepass_columns = ["agent_name","gst_number","short_name", "bill_date","gst_percent","net_amount",'count', 'hsn_code']
    df = pd.DataFrame(list(sale_group_gatepass_obj), columns=sale_group_gatepass_columns)

    default_value=1
    df['gst']=default_value+(df['gst_percent'].astype('float')/100)
    df['cost']=df['net_amount'].astype('float')/df['gst'].astype('float')
    df['total_gst']=df['cost'].astype('float')*(df['gst_percent'].astype('float')/100)
    df['cgst']=(df['total_gst'].astype('float'))/2
    df['sgst']=(df['total_gst'].astype('float'))/2

    df['RATE'] = (df['net_amount'].astype('float')/df['count'].astype('float'))
    df['bill_date'] = pd.to_datetime(df['bill_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime("%Y-%m-%d")
    df['IGST'] = 0
    df['(No column name)'] = 0
    df['INVNO'] = ""
    df = df.rename(columns={'agent_name':'AGENT', 'gst_number':'BTAXNO', 'short_name':'PRODUCT', 'bill_date':'INVDATE','RATE':'RATE','gst_percent':'GST','net_amount':'NETAMOUNT','cgst':'CGST','sgst':'SGST','igst':'IGST', 'hsn_code': 'HSNCODE'})

    final_df_2 = df[['AGENT', 'BTAXNO', 'PRODUCT', 'INVNO','INVDATE','RATE','GST','(No column name)','SGST','CGST','IGST','NETAMOUNT','HSNCODE']]
    final_df_2.round(2)
    final_df_2 = final_df_2.fillna(' ')

    file_name = f"static/media/by_product/monthly_report/{start_date} - {end_date} - GVT MILK GST Bill.xlsx"
    writer = pd.ExcelWriter(file_name, engine="xlsxwriter")
    sheet_name1 = "sheet1"
    sheet_name2 = "sheet2"
    final_df.to_excel(writer, sheet_name=sheet_name1, index=False)
    final_df_2.to_excel(writer, sheet_name=sheet_name2, index=False)
    writer.save()
    document = {}
    document['is_data_available'] = True
    document['file_name'] = f"{start_date} - {end_date} - GVT GST MILK Bill.xlsx"
#     try:
#         image_path = file_name
#         with open(image_path, 'rb') as image_file:
#             encoded_image = b64encode(image_file.read())
#             document['excel'] = encoded_image
#     except Exception as err:
#         print(err)
#     return Response(data=document, status=status.HTTP_200_OK)
# else:
#     document = {}
#     document['is_data_available'] = False
#     return Response(data=document, status=status.HTTP_200_OK)

In [23]:
final_df_2

,AGENT,BTAXNO,PRODUCT,INVNO,INVDATE,RATE,GST,(No column name),SGST,CGST,IGST,NETAMOUNT,HSNCODE
0,K SUMITHA,,Curd 500,,2022-10-01,31.91,0.00,0,0.0,0.0,0,478.65,0403
1,S LATHA,,Curd 500,,2022-10-01,31.91,0.00,0,0.0,0.0,0,127.64,0403
2,S LATHA,,Curd 120,,2022-10-01,8.30,0.00,0,0.0,0.0,0,74.70,0403
3,DURAIKKANNU.P.,33APRPK9938H1Z9,Curd 120,,2022-10-01,8.30,0.00,0,0.0,0.0,0,199.20,0403
4,M.SENTHILKUMAR,,Curd 500,,2022-10-01,31.91,0.00,0,0.0,0.0,0,191.46,0403
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9957,POOBALAKRISHNAN.R,,Curd 500,,2022-10-31,31.91,0.00,0,0.0,0.0,0,382.92,0403
9958,POOBALAKRISHNAN.R,,Curd 120,,2022-10-31,8.30,0.00,0,0.0,0.0,0,415.00,0403
9959,SUGAPRIYA.V,,Curd 120,,2022-10-31,8.30,0.00,0,0.0,0.0,0,249.00,0403
9960,BHAVANESHWARI .S,,Curd 500,,2022-10-31,31.91,0.00,0,0.0,0.0,0,95.73,0403


In [1]:
bank_obj=BankTransactionCarryForward.objects.filter(rid=value.rid)
duplicate= list(bank_obj.values_list('rid'))
duplicate

NameError: name 'value' is not defined

In [1]:
for rid_list in BankTransactionCarryForward.objects.values_list('rid',flat=True).distinct():
    print('rid_list',rid_list)
    BankTransactionCarryForward.objects.filter(id__in=BankTransactionCarryForward.objects.filter(rid=rid_list).values_list('id',flat=True)[1:]).delete()

rid_list 2290100015
rid_list 2253100278
rid_list 2143000302
rid_list 20123000162
rid_list 2281600140
rid_list 21103000207
rid_list 2272400162
rid_list 2283000402
rid_list 2272700278
rid_list 2272900116
rid_list 2280700060
rid_list 21123100108
rid_list 2253000334
rid_list 2263000162
rid_list 2270500278
rid_list 2272100073
rid_list 2280300174
rid_list 2280200310
rid_list 2282800052
rid_list 2290100357
rid_list 2271300329
rid_list 2281400008
rid_list 2281600361
rid_list 2272000324
rid_list 2290600025
rid_list 2281000365
rid_list 2282800201
rid_list 20103000019
rid_list 2233000280
rid_list 2270500366
rid_list 2243000204
rid_list 2272800420
rid_list 21103000175
rid_list 2143000209
rid_list 2270800089
rid_list 2281300317
rid_list 2281500057
rid_list 2182900117
rid_list 2272700458
rid_list 2281500345
rid_list 2282900322
rid_list 2271100101
rid_list 2280600611
rid_list 2182700012
rid_list 2270800182
rid_list 2282800485
rid_list 2270900287
rid_list 2142900011
rid_list 2122600390
rid_list 224290

rid_list 2272300012
rid_list 2282700171
rid_list 2270300360
rid_list 2282300181
rid_list 2213000125
rid_list 2113000030
rid_list 21103100219
rid_list 2271800120
rid_list 21123100572
rid_list 2280600670
rid_list 2282600156
rid_list 2272800411
rid_list 2271300074
rid_list 2282900352
rid_list 2273100018
rid_list 2122500005
rid_list 2271200001
rid_list 2282600419
rid_list 2271300123
rid_list 2153100282
rid_list 2282300418
rid_list 2270300135
rid_list 2222500734
rid_list 21112900003
rid_list 2282100162
rid_list 21103100211
rid_list 2122600153
rid_list 2153000260
rid_list 2272700118
rid_list 2282800250
rid_list 2280400214
rid_list 20113000010
rid_list 2271200291
rid_list 2281900404
rid_list 2122700211
rid_list 2113100075
rid_list 2162900196
rid_list 2270500230
rid_list 2233100236
rid_list 2281800349
rid_list 2242900078
rid_list 2281900327
rid_list 2272700489
rid_list 2243000185
rid_list 2270200371
rid_list 2282600122
rid_list 2280100238
rid_list 2183000026
rid_list 2270200136
rid_list 218300

rid_list 2112900166
rid_list 2270700113
rid_list 2271800353
rid_list 2290100030
rid_list 2113100020
rid_list 2122600117
rid_list 2281700206
rid_list 2272600132
rid_list 2281200295
rid_list 2280200301
rid_list 2270400234
rid_list 2290400027
rid_list 2290100375
rid_list 2283000487
rid_list 2182700362
rid_list 2282200077
rid_list 2083100008
rid_list 2281600129
rid_list 21123100451
rid_list 2282300003
rid_list 2153000130
rid_list 2242900490
rid_list 2270500317
rid_list 2290300101
rid_list 2271500122
rid_list 2280400344
rid_list 21123000396
rid_list 2281000044
rid_list 2281400143
rid_list 2153100110
rid_list 2153000190
rid_list 2183000154
rid_list 2271800283
rid_list 2262900090
rid_list 2233000031
rid_list 21103000102
rid_list 2271600061
rid_list 2280700064
rid_list 2280300239
rid_list 2283100010
rid_list 2273100099
rid_list 2270900332
rid_list 2290400379
rid_list 2272900147
rid_list 2272500441
rid_list 2280700260
rid_list 21112900158
rid_list 2282600252
rid_list 20123000174
rid_list 228280

rid_list 2281600287
rid_list 2222600561
rid_list 2280100168
rid_list 2280700036
rid_list 2173000273
rid_list 2253000409
rid_list 2282600145
rid_list 2253100418
rid_list 2233100213
rid_list 2281200192
rid_list 2282100266
rid_list 21123000167
rid_list 2280600617
rid_list 2173100026
rid_list 2122500088
rid_list 2271800109
rid_list 21103100167
rid_list 2253000363
rid_list 2270700306
rid_list 2282100519
rid_list 2290500143
rid_list 2262900087
rid_list 2290600030
rid_list 2282000282
rid_list 2272100058
rid_list 2281000057
rid_list 2173000145
rid_list 2133000132
rid_list 2270700062
rid_list 2272400281
rid_list 2173100293
rid_list 2233000083
rid_list 2280700192
rid_list 2272600672
rid_list 2281000226
rid_list 2282600091
rid_list 2263000107
rid_list 2183000009
rid_list 20112900192
rid_list 2281000236
rid_list 21103000082
rid_list 21102900071
rid_list 20113000050
rid_list 2273100212
rid_list 2271000196
rid_list 2282700337
rid_list 2272700064
rid_list 2281900082
rid_list 2233000351
rid_list 21123

rid_list 2271800128
rid_list 2142900150
rid_list 2271100340
rid_list 2093000125
rid_list 2281500030
rid_list 2273000054
rid_list 21113000001
rid_list 2173100194
rid_list 2282700259
rid_list 2153100059
rid_list 2270800375
rid_list 2281200354
rid_list 2282700402
rid_list 2122700239
rid_list 2163000112
rid_list 2282100480
rid_list 2213100341
rid_list 2163000332
rid_list 2182900124
rid_list 2222500063
rid_list 2233100375
rid_list 20112600096
rid_list 2253100185
rid_list 21103100285
rid_list 2272100184
rid_list 21103100303
rid_list 20102900055
rid_list 2163000069
rid_list 2280800157
rid_list 2271100032
rid_list 2233000319
rid_list 2143000242
rid_list 2182900234
rid_list 2291000150
rid_list 20123000069
rid_list 2222800537
rid_list 2280500509
rid_list 2281200268
rid_list 2183000301
rid_list 2282000136
rid_list 2281500322
rid_list 2263000367
rid_list 2213100155
rid_list 2272200292
rid_list 2270500315
rid_list 2280900179
rid_list 2272700293
rid_list 2270100214
rid_list 2213000255
rid_list 22706

...

rid_list 2280300340
rid_list 2133100216
rid_list 2270500309
rid_list 2271500276
rid_list 2153100253
rid_list 2112900198
rid_list 2253000506
rid_list 2280300301
rid_list 21102900013
rid_list 2280400313
rid_list 2280200265
rid_list 21103100132
rid_list 2283100087
rid_list 2280400092
rid_list 2122600357
rid_list 2093000095
rid_list 2222800585
rid_list 2122800053
rid_list 2271300058
rid_list 20112800074
rid_list 2272500043
rid_list 2272200077
rid_list 2282500186
rid_list 2281100087
rid_list 20103100074
rid_list 2270500384
rid_list 2270400273
rid_list 2283100069
rid_list 2272800146
rid_list 2282500327
rid_list 2122500149
rid_list 2290100045
rid_list 2242900006
rid_list 2272900381
rid_list 20112800206
rid_list 2143000195
rid_list 2271700156
rid_list 2271900550
rid_list 2280300457
rid_list 2271500282
rid_list 2222700115
rid_list 2281000283
rid_list 2282600232
rid_list 2282200180
rid_list 2282000260
rid_list 21123100205
rid_list 2153000147
rid_list 2183100243
rid_list 2270500016
rid_list 22812

rid_list 2281100103
rid_list 2272800426
rid_list 2270900199
rid_list 2271400181
rid_list 2122500051
rid_list 2282200411
rid_list 2280400257
rid_list 2122700132
rid_list 2183100081
rid_list 2281900083
rid_list 2282200438
rid_list 2280100023
rid_list 2272400327
rid_list 2270600111
rid_list 2280400370
rid_list 2270800333
rid_list 2272700520
rid_list 2113000116
rid_list 2282200065
rid_list 2273000115
rid_list 2282700114
rid_list 2163000074
rid_list 2271700195
rid_list 2122800135
rid_list 20112700174
rid_list 2271100278
rid_list 2153000034
rid_list 2270700341
rid_list 2272600395
rid_list 2271100354
rid_list 2283000470
rid_list 2283100380
rid_list 2281200224
rid_list 2263000312
rid_list 2272800287
rid_list 2271700276
rid_list 2282300396
rid_list 2272900265
rid_list 2271000057
rid_list 2281600366
rid_list 2270300280
rid_list 2281600263
rid_list 2253100009
rid_list 2153100181
rid_list 20102900075
rid_list 2122700025
rid_list 2272600374
rid_list 2122700195
rid_list 2290200400
rid_list 227210023

rid_list 2122500283
rid_list 2271600082
rid_list 2222800211
rid_list 2271700204
rid_list 2233000107
rid_list 2271300304
rid_list 2183100098
rid_list 2272800558
rid_list 2281500055
rid_list 2272800313
rid_list 2233000074
rid_list 20123100130
rid_list 2282300135
rid_list 2183100089
rid_list 2270100198
rid_list 2281400070
rid_list 2272100367
rid_list 2213100189
rid_list 2281900055
rid_list 21123000315
rid_list 2222600725
rid_list 2183000258
rid_list 2183000193
rid_list 2253000006
rid_list 2272800036
rid_list 2290100113
rid_list 21113000216
rid_list 2173100190
rid_list 2270700012
rid_list 2280900019
rid_list 2270900162
rid_list 2273100368
rid_list 2281300353
rid_list 2243000463
rid_list 2153100276
rid_list 2282500022
rid_list 2282300213
rid_list 2262900015
rid_list 2283100091
rid_list 2281800262
rid_list 2280400077
rid_list 2290100024
rid_list 2093000116
rid_list 2270900277
rid_list 2270900248
rid_list 2282000192
rid_list 2271300215
rid_list 2272500563
rid_list 2093000136
rid_list 22831002

rid_list 2263000069
rid_list 2281800296
rid_list 2271600350
rid_list 2162600043
rid_list 2243000109
rid_list 2282200423
rid_list 2281800109
rid_list 20113000199
rid_list 2163000062
rid_list 2173000137
rid_list 2182700311
rid_list 2271300137
rid_list 2133100097
rid_list 2272300285
rid_list 2280100244
rid_list 2253000375
rid_list 2290100064
rid_list 2272900165
rid_list 2182900084
rid_list 2271400194
rid_list 2183000189
rid_list 2153000003
rid_list 2272400346
rid_list 2233000150
rid_list 21102900163
rid_list 20113000101
rid_list 2272300220
rid_list 2272600523
rid_list 2270500159
rid_list 2272000233
rid_list 20123000014
rid_list 2280700120
rid_list 2270700349
rid_list 2262900089
rid_list 2163000091
rid_list 2280100121
rid_list 2113100061
rid_list 2122800226
rid_list 2152800112
rid_list 2272500088
rid_list 2282400463
rid_list 2143000329
rid_list 2290100267
rid_list 2253100120
rid_list 2242900025
rid_list 2273000374
rid_list 2133100197
rid_list 2182800180
rid_list 2271500326
rid_list 2272900

rid_list 2273100158
rid_list 2272900236
rid_list 2270900251
rid_list 2280200353
rid_list 2222600707
rid_list 2271400059
rid_list 2122700175
rid_list 2271400091
rid_list 20113000120
rid_list 2282200171
rid_list 2282300155
rid_list 2281700006
rid_list 2272000170
rid_list 2271800310
rid_list 2113000156
rid_list 2243000465
rid_list 2213000449
rid_list 20112700150
rid_list 20112700242
rid_list 2152900055
rid_list 21102900160
rid_list 2280500956
rid_list 2281100375
rid_list 2222500901
rid_list 2272200271
rid_list 2273100262
rid_list 2281000077
rid_list 2290300141
rid_list 2281000159
rid_list 2271600295
rid_list 2272400203
rid_list 20123100082
rid_list 2282500387
rid_list 2233000323
rid_list 2280700242
rid_list 2282100343
rid_list 2281900093
rid_list 2271200345
rid_list 2271500097
rid_list 2233100080
rid_list 2162900145
rid_list 2283000446
rid_list 2271700141
rid_list 2281400195
rid_list 2253100196
rid_list 2270600182
rid_list 2272700232
rid_list 2290100328
rid_list 2270400158
rid_list 227140

In [2]:
for rid_list in BankTransactionCarryForward.objects.values_list('rid',flat=True).distinct():
    print('rid_list',rid_list)
    BankTransactionCarryForward.objects.filter(id__in=BankTransactionCarryForward.objects.filter(rid=rid_list).values_list('id',flat=True)[1:]).delete()

rid_list 2290100015
rid_list 2253100278
rid_list 2143000302
rid_list 20123000162
rid_list 2281600140
rid_list 21103000207
rid_list 2272400162
rid_list 2283000402
rid_list 2272700278
rid_list 2272900116
rid_list 2280700060
rid_list 21123100108
rid_list 2253000334
rid_list 2263000162
rid_list 2270500278
rid_list 2272100073
rid_list 2280300174
rid_list 2280200310
rid_list 2282800052
rid_list 2290100357
rid_list 2271300329
rid_list 2281400008
rid_list 2281600361
rid_list 2272000324
rid_list 2290600025
rid_list 2281000365
rid_list 2282800201
rid_list 20103000019
rid_list 2233000280
rid_list 2270500366
rid_list 2243000204
rid_list 2272800420
rid_list 21103000175
rid_list 2143000209
rid_list 2270800089
rid_list 2281300317
rid_list 2281500057
rid_list 2182900117
rid_list 2272700458
rid_list 2281500345
rid_list 2282900322
rid_list 2271100101
rid_list 2280600611
rid_list 2182700012
rid_list 2270800182
rid_list 2282800485
rid_list 2270900287
rid_list 2142900011
rid_list 2122600390
rid_list 224290

rid_list 2282600156
rid_list 2272800411
rid_list 2271300074
rid_list 2282900352
rid_list 2273100018
rid_list 2122500005
rid_list 2271200001
rid_list 2282600419
rid_list 2271300123
rid_list 2153100282
rid_list 2282300418
rid_list 2270300135
rid_list 2222500734
rid_list 21112900003
rid_list 2282100162
rid_list 21103100211
rid_list 2122600153
rid_list 2153000260
rid_list 2272700118
rid_list 2282800250
rid_list 2280400214
rid_list 20113000010
rid_list 2271200291
rid_list 2281900404
rid_list 2122700211
rid_list 2113100075
rid_list 2162900196
rid_list 2270500230
rid_list 2233100236
rid_list 2281800349
rid_list 2242900078
rid_list 2281900327
rid_list 2272700489
rid_list 2243000185
rid_list 2270200371
rid_list 2282600122
rid_list 2280100238
rid_list 2183000026
rid_list 2270200136
rid_list 2183000147
rid_list 2272500098
rid_list 21113000134
rid_list 2282600269
rid_list 21102900030
rid_list 2272100048
rid_list 2222500868
rid_list 2272100209
rid_list 2222500140
rid_list 2222600123
rid_list 227170

rid_list 2282300003
rid_list 2153000130
rid_list 2242900490
rid_list 2270500317
rid_list 2290300101
rid_list 2271500122
rid_list 2280400344
rid_list 21123000396
rid_list 2281000044
rid_list 2281400143
rid_list 2153100110
rid_list 2153000190
rid_list 2183000154
rid_list 2271800283
rid_list 2262900090
rid_list 2233000031
rid_list 21103000102
rid_list 2271600061
rid_list 2280700064
rid_list 2280300239
rid_list 2283100010
rid_list 2273100099
rid_list 2270900332
rid_list 2290400379
rid_list 2272900147
rid_list 2272500441
rid_list 2280700260
rid_list 21112900158
rid_list 2282600252
rid_list 20123000174
rid_list 2282800426
rid_list 2282500176
rid_list 2173000267
rid_list 2270500055
rid_list 2272400071
rid_list 20112900184
rid_list 2282100056
rid_list 2281200184
rid_list 2173100279
rid_list 2282300071
rid_list 2153100113
rid_list 2271200118
rid_list 2272400093
rid_list 2280400179
rid_list 2152900016
rid_list 20103000099
rid_list 2272800256
rid_list 2270300044
rid_list 2281300319
rid_list 21529

rid_list 2182900132
rid_list 2281500224
rid_list 2282200335
rid_list 2272300362
rid_list 2290100157
rid_list 2282200069
rid_list 2122800029
rid_list 2283000344
rid_list 2282400453
rid_list 2281900153
rid_list 2182700029
rid_list 2270400097
rid_list 2132700147
rid_list 2271000296
rid_list 2271600136
rid_list 2270400034
rid_list 2270600250
rid_list 2283000556
rid_list 2272700312
rid_list 2272500058
rid_list 2281600287
rid_list 2222600561
rid_list 2280100168
rid_list 2280700036
rid_list 2173000273
rid_list 2253000409
rid_list 2282600145
rid_list 2253100418
rid_list 2233100213
rid_list 2281200192
rid_list 2282100266
rid_list 21123000167
rid_list 2280600617
rid_list 2173100026
rid_list 2122500088
rid_list 2271800109
rid_list 21103100167
rid_list 2253000363
rid_list 2270700306
rid_list 2282100519
rid_list 2290500143
rid_list 2262900087
rid_list 2290600030
rid_list 2282000282
rid_list 2272100058
rid_list 2281000057
rid_list 2173000145
rid_list 2133000132
rid_list 2270700062
rid_list 227240028

rid_list 2142900150
rid_list 2271100340
rid_list 2093000125
rid_list 2281500030
rid_list 2273000054
rid_list 21113000001
rid_list 2173100194
rid_list 2282700259
rid_list 2153100059
rid_list 2270800375
rid_list 2281200354
rid_list 2282700402
rid_list 2122700239
rid_list 2163000112
rid_list 2282100480
rid_list 2213100341
rid_list 2163000332
rid_list 2182900124
rid_list 2222500063
rid_list 2233100375
rid_list 20112600096
rid_list 2253100185
rid_list 21103100285
rid_list 2272100184
rid_list 21103100303
rid_list 20102900055
rid_list 2163000069
rid_list 2280800157
rid_list 2271100032
rid_list 2233000319
rid_list 2143000242
rid_list 2182900234
rid_list 2291000150
rid_list 20123000069
rid_list 2222800537
rid_list 2280500509
rid_list 2281200268
rid_list 2183000301
rid_list 2282000136
rid_list 2281500322
rid_list 2263000367
rid_list 2213100155
rid_list 2272200292
rid_list 2270500315
rid_list 2280900179
rid_list 2272700293
rid_list 2270100214
rid_list 2213000255
rid_list 2270600165
rid_list 22913

...

rid_list 2282700223
rid_list 20103000110
rid_list 2281900135
rid_list 21103100080
rid_list 2290200042
rid_list 2270300243
rid_list 2233100161
rid_list 2272400069
rid_list 2272300239
rid_list 2273100010
rid_list 2290300161
rid_list 2281600041
rid_list 21103000236
rid_list 2282800156
rid_list 2271000388
rid_list 2270200196
rid_list 2183000088
rid_list 2270600253
rid_list 2272800488
rid_list 2272900141
rid_list 2281700211
rid_list 2272400244
rid_list 2280900375
rid_list 2183000068
rid_list 2270700087
rid_list 2133000014
rid_list 20112800002
rid_list 2253000472
rid_list 2280900207
rid_list 2270600071
rid_list 2280700098
rid_list 2283000492
rid_list 2270300268
rid_list 2270600121
rid_list 2280200198
rid_list 2270300026
rid_list 2282500479
rid_list 2281200055
rid_list 2272500514
rid_list 2273100049
rid_list 2162900194
rid_list 2153100115
rid_list 21123100225
rid_list 2280800246
rid_list 20123100134
rid_list 2282900282
rid_list 2290200187
rid_list 2272600295
rid_list 2271200312
rid_list 22707

rid_list 2282400022
rid_list 2273000290
rid_list 2183100057
rid_list 2122600084
rid_list 2272900028
rid_list 2282600010
rid_list 2270100184
rid_list 2271800305
rid_list 2233100395
rid_list 21113000238
rid_list 2272400487
rid_list 2122800258
rid_list 2271200216
rid_list 2270400171
rid_list 2122500050
rid_list 2272400337
rid_list 2270100378
rid_list 2173100256
rid_list 2281800119
rid_list 20112800198
rid_list 2213000253
rid_list 2281400298
rid_list 2280800175
rid_list 2280500796
rid_list 2272000402
rid_list 20112600152
rid_list 2133100049
rid_list 2253000294
rid_list 2270200121
rid_list 2280500128
rid_list 2271600180
rid_list 2272500343
rid_list 2271700306
rid_list 21103100103
rid_list 2212900050
rid_list 2233000251
rid_list 2271000266
rid_list 2222800426
rid_list 2282300042
rid_list 2282600267
rid_list 2273100244
rid_list 2271200230
rid_list 2272100233
rid_list 2272800033
rid_list 2122500168
rid_list 2271600311
rid_list 2143000217
rid_list 2281400107
rid_list 2282400390
rid_list 2282800

rid_list 2272600627
rid_list 2113100196
rid_list 2272500129
rid_list 21123000301
rid_list 2253100195
rid_list 2282900085
rid_list 2282500561
rid_list 20112900111
rid_list 2282600251
rid_list 2282600343
rid_list 2281600386
rid_list 2272000002
rid_list 2282700395
rid_list 2270400382
rid_list 2233000073
rid_list 2122700280
rid_list 2280400252
rid_list 2271900105
rid_list 2183000097
rid_list 2272400401
rid_list 2282700236
rid_list 2270800224
rid_list 2272900341
rid_list 2122600328
rid_list 2213100130
rid_list 20112900161
rid_list 2272100112
rid_list 2290100317
rid_list 2282700109
rid_list 2243000060
rid_list 21123100044
rid_list 2280200358
rid_list 21112900281
rid_list 2262900080
rid_list 2291300046
rid_list 2282700490
rid_list 2281100356
rid_list 2272100045
rid_list 21113000170
rid_list 20123100132
rid_list 2122800300
rid_list 2280100053
rid_list 2253100284
rid_list 2182900135
rid_list 2280500055
rid_list 2280900083
rid_list 2280500817
rid_list 2183100117
rid_list 21102800549
rid_list 227

rid_list 2281000082
rid_list 2243000254
rid_list 2282300401
rid_list 2122700177
rid_list 2280400205
rid_list 2272500290
rid_list 2290300443
rid_list 2133100131
rid_list 2283100244
rid_list 2182700361
rid_list 2280300124
rid_list 2233100079
rid_list 2271000305
rid_list 2271000017
rid_list 2271800319
rid_list 2282000189
rid_list 2282600499
rid_list 2282600233
rid_list 2182800043
rid_list 2271200181
rid_list 2272100208
rid_list 2272000354
rid_list 2272200263
rid_list 2272800177
rid_list 2270300317
rid_list 2272200185
rid_list 2213100431
rid_list 21103100182
rid_list 2271700424
rid_list 2281200213
rid_list 21113000286
rid_list 2272400436
rid_list 2280900348
rid_list 2272300381
rid_list 2281100247
rid_list 2213100419
rid_list 2280700325
rid_list 2122500076
rid_list 2272700013
rid_list 2270100321
rid_list 2270100149
rid_list 2182800224
rid_list 2153000276
rid_list 2222500611
rid_list 2270800253
rid_list 2282800176
rid_list 2183100045
rid_list 2133000268
rid_list 2271400099
rid_list 228220037

rid_list 2271500194
rid_list 2242900045
rid_list 2083000037
rid_list 2282100380
rid_list 2281200219
rid_list 2272200186
rid_list 2270900035
rid_list 2271200214
rid_list 2290100073
rid_list 2282600311
rid_list 2270900118
rid_list 2281900282
rid_list 2270500120
rid_list 2242900181
rid_list 2280800193
rid_list 2280500337
rid_list 2282100509
rid_list 2233000079
rid_list 2233000125
rid_list 2271000224
rid_list 20112600118
rid_list 2270600081
rid_list 2281600070
rid_list 2222800117
rid_list 2272000300
rid_list 2272200336
rid_list 2272400213
rid_list 2270700165
rid_list 2271400084
rid_list 2222600821
rid_list 21103100280
rid_list 2270100288
rid_list 2281800319
rid_list 2281900403
rid_list 21103000055
rid_list 2222500774
rid_list 2282800038
rid_list 2262900070
rid_list 2281000351
rid_list 2281400226
rid_list 2281600122
rid_list 2282700369
rid_list 2283000634
rid_list 2271200083
rid_list 2083000069
rid_list 2242900031
rid_list 2282300300
rid_list 2281300156
rid_list 2290100021
rid_list 22721003

In [63]:
obj=set(Sale.objects.filter(product_id=10,sale_group__date__gte='2022-04-01',sale_group__date__lte='2022-11-17').values_list('id', flat= True))
obj
object = set(Product.objects.filter(is_active=True).values_list('id', flat= True))
obj

{3820353}

In [64]:
objs= object-obj
objs
x=Product.objects.filter(id__in=objs).values_list('id','name' )
x

<QuerySet [(23, 'FCM'), (24, 'SM'), (1, 'Tonned'), (3, 'STD'), (25, 'Curd'), (10, 'Cup curd'), (28, 'BMJF'), (8, 'Butter Milk'), (26, 'BM Pet Jar'), (7, 'Curd'), (2, 'STD'), (4, 'FCM'), (6, 'FCM'), (30, 'Curd'), (32, 'TEAMATE'), (31, 'TEAMATE')]>

In [66]:
# for rid_list in BankTransactionCarryForward.objects.values_list('rid',flat=True).distinct():
#     print('rid_list',rid_list)
BankTransactionCarryForward.objects.filter(rid="2272600055").values_list('id',flat=True)[1:]).delete()

(0, {'transaction.BankTransactionCarryForward': 0})

In [1]:
for rid_list in BankTransactionCarryForward.objects.filter(updated_by_id=25183).values_list('rid',flat=True).distinct():
    print('rid_list',rid_list)
    BankTransactionCarryForward.objects.filter(id__in=BankTransactionCarryForward.objects.filter(rid=rid_list,updated_by_id=25183).values_list('id',flat=True)[1:]).delete()

rid_list 2282200309
rid_list 2271700406
rid_list 2253100278
rid_list 2281600140
rid_list 2272700197
rid_list 2270800062
rid_list 2272400162
rid_list 2272700278
rid_list 2272900116
rid_list 2253000334
rid_list 2270500278
rid_list 2272100073
rid_list 2270600157
rid_list 2271300329
rid_list 2272500037
rid_list 2272000324
rid_list 2273100196
rid_list 2271500051
rid_list 2282600107
rid_list 2271800025
rid_list 2282700487
rid_list 2233000280
rid_list 2270500366
rid_list 2243000204
rid_list 2272800420
rid_list 2233000072
rid_list 2270800390
rid_list 2270200257
rid_list 2270800089
rid_list 2280900353
rid_list 2270900355
rid_list 2272700458
rid_list 2271100101
rid_list 2270800182
rid_list 2282800485
rid_list 2271400036
rid_list 2270900287
rid_list 2272000111
rid_list 2242900180
rid_list 2271600222
rid_list 2270700251
rid_list 2273100247
rid_list 2282600490
rid_list 2233000365
rid_list 2233000380
rid_list 2271000102
rid_list 2243000305
rid_list 2280400078
rid_list 2270800125
rid_list 2272200146


rid_list 2272500589
rid_list 2282300111
rid_list 2253000033
rid_list 2270500007
rid_list 2271300034
rid_list 2233000231
rid_list 2271100259
rid_list 2282500004
rid_list 2272600301
rid_list 2272800160
rid_list 2271800068
rid_list 2272100394
rid_list 2253000211
rid_list 2270500068
rid_list 2273000275
rid_list 2271900406
rid_list 2271200076
rid_list 2272300265
rid_list 2270800339
rid_list 2272800495
rid_list 2271500157
rid_list 2272000272
rid_list 2271100157
rid_list 2271800201
rid_list 2271800359
rid_list 2253000754
rid_list 2243000323
rid_list 2272800159
rid_list 2272600361
rid_list 2282300408
rid_list 2271800019
rid_list 2233000282
rid_list 2280800132
rid_list 2253100415
rid_list 2270500234
rid_list 2282700112
rid_list 2270200284
rid_list 2270500313
rid_list 2272200264
rid_list 2270700047
rid_list 2271500083
rid_list 2270700167
rid_list 2270500017
rid_list 2270600011
rid_list 2270500224
rid_list 2270600149
rid_list 2270100259
rid_list 2271700259
rid_list 2271800020
rid_list 2273000192


rid_list 2270300081
rid_list 2272800412
rid_list 2272100167
rid_list 2270300187
rid_list 2233100076
rid_list 2271300327
rid_list 2270800094
rid_list 2243000366
rid_list 2233000045
rid_list 2271900522
rid_list 2233000341
rid_list 2272600269
rid_list 2243000388
rid_list 2271900332
rid_list 2271100144
rid_list 2270200242
rid_list 2272600377
rid_list 2272700162
rid_list 2253100102
rid_list 2270500221
rid_list 2233100055
rid_list 2270400385
rid_list 2253100466
rid_list 2282300456
rid_list 2270700246
rid_list 2233100276
rid_list 2271700222
rid_list 2271600192
rid_list 2270100105
rid_list 2271000328
rid_list 2270700042
rid_list 2253100114
rid_list 2272200346
rid_list 2271300236
rid_list 2272000369
rid_list 2273000179
rid_list 2273100052
rid_list 2271700068
rid_list 2271200071
rid_list 2243000165
rid_list 2270700180
rid_list 2273100258
rid_list 2270400039
rid_list 2273000183
rid_list 2243000321
rid_list 2280900181
rid_list 2233000290
rid_list 2272800141
rid_list 2282300143
rid_list 2270800270


rid_list 2271900116
rid_list 2271900283
rid_list 2270800310
rid_list 2272500081
rid_list 2270700183
rid_list 2271500197
rid_list 2270600193
rid_list 2270800188
rid_list 2273000119
rid_list 2282400439
rid_list 2271400129
rid_list 2280900383
rid_list 2271100364
rid_list 2233000066
rid_list 2270500291
rid_list 2242900008
rid_list 2270800197
rid_list 2270500028
rid_list 2270100346
rid_list 2271100316
rid_list 2271200236
rid_list 2273100130
rid_list 2272500062
rid_list 2272800073
rid_list 2272700253
rid_list 2271200091
rid_list 2270300383
rid_list 2272500249
rid_list 2271800167
rid_list 2272800306
rid_list 2272500124
rid_list 2272000061
rid_list 2270700048
rid_list 2270600254
rid_list 2271600236
rid_list 2282200107
rid_list 2271200276
rid_list 2271800333
rid_list 2271200226
rid_list 2271200173
rid_list 2233000270
rid_list 2272200029
rid_list 2273100190
rid_list 2272500643
rid_list 2271400096
rid_list 2282600462
rid_list 2272600159
rid_list 2270300099
rid_list 2271700256
rid_list 2272700186


rid_list 2272200116
rid_list 2271500341
rid_list 2272500114
rid_list 2271100181
rid_list 2272800170
rid_list 2272500617
rid_list 2270300308
rid_list 2273100021
rid_list 2270700270
rid_list 2272600659
rid_list 2272100185
rid_list 2282500155
rid_list 2282400082
rid_list 2253000637
rid_list 2271800180
rid_list 2270400224
rid_list 2272600375
rid_list 2270300096
rid_list 2243000464
rid_list 2272800303
rid_list 2233100214
rid_list 2271800046
rid_list 2270500368
rid_list 2270600189
rid_list 2272500236
rid_list 2280700123
rid_list 2271000263
rid_list 2282300409
rid_list 2272400220
rid_list 2270500010
rid_list 2270500192
rid_list 2272300392
rid_list 2253000806
rid_list 2282800132
rid_list 2271500218
rid_list 2272700297
rid_list 2272800196
rid_list 2271800171
rid_list 2272700001
rid_list 2271900451
rid_list 2271100036
rid_list 2271400003
rid_list 2271500045
rid_list 2271400154
rid_list 2270700039
rid_list 2271000041
rid_list 2270800271
rid_list 2272900041
rid_list 2272200007
rid_list 2272900133


...

rid_list 2233100217
rid_list 2253100451
rid_list 2281700001
rid_list 2272800520
rid_list 2233100399
rid_list 2272300118
rid_list 2272900145
rid_list 2271100023
rid_list 2270700058
rid_list 2272400175
rid_list 2280700348
rid_list 2273100361
rid_list 2272400234
rid_list 2271700328
rid_list 2273000320
rid_list 2272300397
rid_list 2233000167
rid_list 2233100187
rid_list 2272100320
rid_list 2271400144
rid_list 2270100036
rid_list 2272300415
rid_list 2243000008
rid_list 2243000050
rid_list 2271200070
rid_list 2233100278
rid_list 2271400119
rid_list 2272600422
rid_list 2272300054
rid_list 2282800236
rid_list 2271000038
rid_list 2271200178
rid_list 2270400073
rid_list 2273100141
rid_list 2273100120
rid_list 2243000353
rid_list 2270200213
rid_list 2272600203
rid_list 2273100333
rid_list 2271900498
rid_list 2273100093
rid_list 2271700169
rid_list 2273000149
rid_list 2282400144
rid_list 2271500012
rid_list 2270800246
rid_list 2270800047
rid_list 2272800052
rid_list 2233100418
rid_list 2243000432


rid_list 2272500515
rid_list 2271200153
rid_list 2243000100
rid_list 2271500287
rid_list 2271300218
rid_list 2271200184
rid_list 2280600312
rid_list 2271500203
rid_list 2272400396
rid_list 2271700182
rid_list 2272700020
rid_list 2253100316
rid_list 2272700026
rid_list 2271600329
rid_list 2271300183
rid_list 2272200138
rid_list 2270400263
rid_list 2272700126
rid_list 2271500329
rid_list 2273100006
rid_list 2271700316
rid_list 2233100384
rid_list 2281200146
rid_list 2271400004
rid_list 2253100008
rid_list 2270200168
rid_list 2272700084
rid_list 2270600005
rid_list 2270500167
rid_list 2282500500
rid_list 2270400361
rid_list 2233000292
rid_list 2272000383
rid_list 2272600674
rid_list 2272100398
rid_list 2271000386
rid_list 2271500295
rid_list 2272000149
rid_list 2271700119
rid_list 2271100054
rid_list 2270300386
rid_list 2273000038
rid_list 2270600138
rid_list 2272900158
rid_list 2282600006
rid_list 2271000404
rid_list 2233100263
rid_list 2272300142
rid_list 2273000290
rid_list 2272000034


rid_list 2272100112
rid_list 2282700109
rid_list 2271500198
rid_list 2272300271
rid_list 2243000060
rid_list 2270600361
rid_list 2282700490
rid_list 2272100045
rid_list 2270200245
rid_list 2270200012
rid_list 2280100053
rid_list 2253100284
rid_list 2270100215
rid_list 2270600230
rid_list 2273100159
rid_list 2270400188
rid_list 2242900002
rid_list 2271800284
rid_list 2272700277
rid_list 2270900341
rid_list 2233100092
rid_list 2270400059
rid_list 2280200122
rid_list 2253000010
rid_list 2270700215
rid_list 2270200104
rid_list 2272800524
rid_list 2272200339
rid_list 2272100270
rid_list 2270100185
rid_list 2272500091
rid_list 2271300259
rid_list 2272900013
rid_list 2270600019
rid_list 2272100319
rid_list 2272300115
rid_list 2270100273
rid_list 2271600082
rid_list 2271700204
rid_list 2233000107
rid_list 2270600004
rid_list 2271300304
rid_list 2272600623
rid_list 2271700273
rid_list 2272800558
rid_list 2270600085
rid_list 2271600044
rid_list 2272800313
rid_list 2233000074
rid_list 2282300135


rid_list 2270400302
rid_list 2270600205
rid_list 2272400407
rid_list 2273000082
rid_list 2271300099
rid_list 2270500186
rid_list 2272800380
rid_list 2270900231
rid_list 2272600696
rid_list 2233100234
rid_list 2271600189
rid_list 2242900206
rid_list 2280300146
rid_list 2270100286
rid_list 2270400214
rid_list 2270900120
rid_list 2272100096
rid_list 2270200087
rid_list 2270200107
rid_list 2282600278
rid_list 2272800345
rid_list 2273000380
rid_list 2271200084
rid_list 2233100195
rid_list 2270100179
rid_list 2271500018
rid_list 2233100135
rid_list 2272500325
rid_list 2271300001
rid_list 2270500244
rid_list 2272000201
rid_list 2271100224
rid_list 2273000015
rid_list 2272400395
rid_list 2272700169
rid_list 2272200112
rid_list 2271300024
rid_list 2242900470
rid_list 2282000381
rid_list 2270900342
rid_list 2272000160
rid_list 2271600018
rid_list 2273100095
rid_list 2272900180
rid_list 2233100304
rid_list 2272600619
rid_list 2272700471
rid_list 2282700412
rid_list 2271000376
rid_list 2272600133


rid_list 2272800453
rid_list 2272800161
rid_list 2282000185
rid_list 2253100225
rid_list 2272000204
rid_list 2271000172
rid_list 2272000128
rid_list 2272100278
rid_list 2271300309
rid_list 2272800074
rid_list 2272100177
rid_list 2272700355
rid_list 2272200065
rid_list 2272100291
rid_list 2272400130
rid_list 2270800329
rid_list 2272100183
rid_list 2272700153
rid_list 2270800170
rid_list 2272900036
rid_list 2271100274
rid_list 2271100292
rid_list 2270600048
rid_list 2270300013
rid_list 2272400467
rid_list 2272700022
rid_list 2272000390
rid_list 2270300080
rid_list 2253000222
rid_list 2271200152
rid_list 2280500115
rid_list 2270900103
rid_list 2273100094
rid_list 2253000206
rid_list 2273100028
rid_list 2233000285
rid_list 2271100206
rid_list 2272100127
rid_list 2271000222
rid_list 2271700397
rid_list 2272700170
rid_list 2272500060
rid_list 2280200155
rid_list 2273000137
rid_list 2272400390
rid_list 2253000079
rid_list 2270300060
rid_list 2270300022
rid_list 2253100458
rid_list 2272400070


In [40]:
import datetime
print(datetime.datetime.now())

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'int'

In [5]:
BusinessWiseProductDiscountTrace.objects.filter(mrp=60).update(mrp=48,discounted_price=46.04,end_date='2022-11-04 05:30:00.000')

36

In [6]:
x=(BusinessWiseProductDiscount.objects.filter(product_id=6).values_list('id',flat=True))

x

<QuerySet [12, 111, 93, 120, 129, 165, 21, 174, 237, 183, 192, 201, 264, 282, 291, 300, 309, 318, 327, 336, '...(remaining elements truncated)...']>

In [7]:
for i in x:
    if BusinessWiseProductDiscountTrace.objects.filter(mrp=48,business_wise_discount_id=i):
        BusinessWiseProductDiscountTrace.objects.create(mrp=60,discounted_price=57.54,start_date='2022-11-05',business_wise_discount_id=i,product_discount_started_by_id=18292,end_date='2070-01-05')


In [1]:
ProductNameAndQuantityTrace.objects.create(start_date='2022-11-23',end_date='2051-05-01',short_name='Curd 5000',product_id=32,unit_id=4,quantity=500)

<ProductNameAndQuantityTrace: ProductNameAndQuantityTrace object>

In [2]:
business_type_id = [1,2,3,4,5,6,7,8,9,10,12,15,16]
for i in business_type_id:
    BusinessProductConcessionMap.objects.create(business_type_id=i,concession_type_id=2,product_id=32),

In [3]:
business_type_id = [1,2,3,9,16]
for i in business_type_id:
    if i==9:
        BusinessTypeWiseProductDiscount.objects.create(business_type_id=i,discounted_price=322.15,product_id=32,created_by_id=1,modified_by_id=1)
    elif i==3:
        BusinessTypeWiseProductDiscount.objects.create(business_type_id=i,discounted_price=390,product_id=32,created_by_id=1,modified_by_id=1)
    else:
        BusinessTypeWiseProductDiscount.objects.create(business_type_id=i,discounted_price=354.36,product_id=32,created_by_id=1,modified_by_id=1)



In [4]:
business_type_wise_discount_id = [44,45,46,47,48]
for i in business_type_wise_discount_id:
    if i == 46:
        BusinessTypeWiseProductDiscountTrace.objects.create(business_type_wise_discount_id=i,start_date='2022-11-23',end_date='2051-05-01',mrp=390,discounted_price=390,product_discount_ended_by_id=18292,product_discount_started_by_id=18292)
    if i == 47:
         BusinessTypeWiseProductDiscountTrace.objects.create(business_type_wise_discount_id=i,start_date='2022-11-23',end_date='2051-05-01',mrp=390,discounted_price=322.15,product_discount_ended_by_id=18292,product_discount_started_by_id=18292)
    else:
         BusinessTypeWiseProductDiscountTrace.objects.create(business_type_wise_discount_id=i,start_date='2022-11-23',end_date='2051-05-01',mrp=390,discounted_price=354.36,product_discount_ended_by_id=18292,product_discount_started_by_id=18292)

/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField BusinessTypeWiseProductDiscountTrace.start_date received a naive datetime (2022-11-23 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField BusinessTypeWiseProductDiscountTrace.end_date received a naive datetime (2051-05-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [5]:
x=list(set(BusinessWiseProductDiscount.objects.filter(product_id__in=[7,10,25]).values_list('business_id',flat=True)))

In [7]:
len(x)


295

In [22]:
a=set(BusinessWiseProductDiscount.objects.filter(product_id=25,business__business_type_id=5).values_list('business_id',flat=True))
a

{1439,
 1440,
 1441,
 1501,
 1513,
 1514,
 1515,
 1517,
 1521,
 1522,
 1525,
 1528,
 1535,
 1538,
 1539,
 1540,
 1542,
 1543,
 1545,
 1549,
 1552,
 1553,
 1554,
 1557,
 1558,
 1559,
 1564,
 1566,
 1571,
 1572,
 1573,
 1575,
 1744,
 1745,
 1746,
 1747,
 1748,
 1749,
 1750,
 1751,
 1752,
 1754,
 1758,
 1759,
 1762,
 1763,
 1765,
 1767,
 1768,
 1769,
 1771,
 1772,
 1773,
 1774,
 1861,
 1947,
 2011,
 2012,
 2014,
 2015,
 2016,
 2018,
 2019,
 2020,
 2021,
 2022,
 2026,
 2027,
 2029,
 2031,
 2034,
 2036,
 2039,
 2040,
 2041,
 2042,
 2044,
 2046,
 2047,
 2050,
 2052,
 2053,
 2054,
 2055,
 2056,
 2058,
 2130,
 2133,
 2134,
 2141,
 2142,
 2143,
 2144,
 2145,
 2146,
 2147,
 2148,
 2149,
 2150,
 2151,
 2152,
 2154,
 2155,
 2156,
 2335,
 2384,
 2389,
 2399,
 2400,
 2404,
 2405,
 2409,
 2410,
 2412,
 2413,
 2414,
 2419,
 2420,
 2421,
 2422,
 2424,
 2425,
 2426,
 2427,
 2430,
 2431,
 2432,
 2526,
 2529,
 2530,
 2531,
 2532,
 2533,
 2534,
 2535,
 2543,
 2548,
 2549,
 2550,
 2745,
 2746,
 2747,
 2779,

In [6]:

for i in x:
    if BusinessWiseProductDiscount.objects.filter(business=i,business__business_type_id=1):
        BusinessWiseProductDiscount.objects.create(discounted_price=354.36, business_id=i, created_by_id=1,modified_by_id=1,product_id=32)
        
    elif BusinessWiseProductDiscount.objects.filter(business=i,business__business_type_id=9):
        BusinessWiseProductDiscount.objects.create(discounted_price=322.15, business_id=i, created_by_id=1,modified_by_id=1,product_id=32)
              
    else:
        BusinessWiseProductDiscount.objects.create(discounted_price=390, business_id=i, created_by_id=1,modified_by_id=1,product_id=32)

In [11]:
m_390=list(BusinessWiseProductDiscount.objects.filter(discounted_price=390).values_list('id',flat=True))
# print(m_390)
m_354=list(BusinessWiseProductDiscount.objects.filter(discounted_price="354.36").values_list('id',flat=True))
# print(m_354)
m_322=list(BusinessWiseProductDiscount.objects.filter(discounted_price="322.15").values_list('id',flat=True))
# print(m_322)

In [2]:
for i in m_390:
    BusinessWiseProductDiscountTrace.objects.create(start_date='2022-11-23',end_date='2051-05-01',mrp=390,discounted_price=390,business_wise_discount_id=i,product_discount_ended_by_id=1,product_discount_started_by_id=18292)

/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField BusinessWiseProductDiscountTrace.start_date received a naive datetime (2022-11-23 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField BusinessWiseProductDiscountTrace.end_date received a naive datetime (2051-05-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [10]:
for i in m_354:
    BusinessWiseProductDiscountTrace.objects.create(start_date='2022-11-23',end_date='2051-05-01',mrp=390,discounted_price=354.36,business_wise_discount_id=i,product_discount_ended_by_id=1,product_discount_started_by_id=18292)
    print('x')

x


In [11]:
for j in m_322:
    BusinessWiseProductDiscountTrace.objects.create(start_date='2022-11-23',end_date='2051-05-01',mrp=390,discounted_price=322.15,business_wise_discount_id=j,product_discount_ended_by_id=1,product_discount_started_by_id=18292)

In [12]:
for i in m_390:
    BusinessWiseProductDiscountTrace.objects.filter(start_date='2022-11-01',end_date='2051-05-01',mrp=390,discounted_price=390,business_wise_discount_id=i,product_discount_ended_by_id=1,product_discount_started_by_id=18292).update(start_date='2022-11-23')

In [13]:
for i in m_354:
    BusinessWiseProductDiscountTrace.objects.filter(start_date='2022-11-01',end_date='2051-05-01',mrp=390,discounted_price=354.36,business_wise_discount_id=i,product_discount_ended_by_id=1,product_discount_started_by_id=18292).update(start_date='2022-11-23')
    print('x')

x


In [14]:
for j in m_322:
    BusinessWiseProductDiscountTrace.objects.filter(start_date='2022-11-01',end_date='2051-05-01',mrp=390,discounted_price=322.15,business_wise_discount_id=j,product_discount_ended_by_id=1,product_discount_started_by_id=18292).update(start_date='2022-11-23')

In [ ]:
def serve_byproduct_analytic_report(request):

    data={
    "from_date":request.data['from_date'],
    "to_date":request.data['to_date'],
    "report_option":request.data['report_option'],
    "type":request.data['type']
    }
    if request.data["type"] == "Sale":
        # print('Check1')
        if request.data['report_option'] == "Booth Wise":
            # booth_codes = [2900]
            # print('check2')
            booth_codes =  request.data['booth_code']
            by_sale_group_booth_obj=BySaleGroup.objects.filter(ordered_date__range=[data["from_date"],data["to_date"]],business__code__in=booth_codes)
            by_sale_group_booth_list=by_sale_group_booth_obj.values_list("id","business__code","business__business_type__name","bysale__by_product__quantity","ordered_date","bysale__by_product__name","bysale__count","bysale__by_product__cgst_amount","bysale__by_product__sgst_amount","bysale__total_cost","bysale__by_product__cgst_percent","bysale__by_product__sgst_percent")
            by_sale_group_booth_column=["id","business_code","business_type","quantity","ordered_date","by_product_name","bysale_count","cgst_amount","sgst_amount","total_cost","cgst_percent","sgst_percent"]
            by_sale_group_booth_df=pd.DataFrame(by_sale_group_booth_list,columns=by_sale_group_booth_column)
            # print('check')
            by_sale_group_booth_df['GST%'] = by_sale_group_booth_df['cgst_percent']+by_sale_group_booth_df['sgst_percent']
            by_sale_group_booth_df.drop('cgst_percent', inplace=True, axis=1)
            by_sale_group_booth_df.drop('sgst_percent', inplace=True, axis=1)
            by_sale_group_booth_df.drop('id', inplace=True, axis=1)
            
            agent_booth_obj=BusinessAgentMap.objects.filter(business__code__in=booth_codes).values_list('agent__first_name',"business__code")
            agent_booth_column=['agent_name',"business_code"]
            agent_booth_df=pd.DataFrame(agent_booth_obj,columns=agent_booth_column)
            
            booth_type_df = pd.merge(by_sale_group_booth_df, agent_booth_df, left_on='business_code', right_on='business_code', how='left')
            
            booth_type_df.insert(0, 'S.No', range(1, 1 + len(booth_type_df)))
            
            booth_type_df = booth_type_df.reindex(columns=[
            "S.No","business_code",'agent_name',"business_type","quantity","ordered_date","by_product_name",
            'GST%',"bysale_count","cgst_amount","sgst_amount","total_cost"])
        
            
            booth_type_df = booth_type_df.rename(columns={'S.No': 'S.No', 'business_code': 'Booth No','agent_name':'Agent Name','business_type':'Business Type','quantity' :'Product Quantity','ordered_date': 'Date','by_product_name':'Product Name','bysale_count':'Product Count','GST%':' Product GST%','cgst_amount':'Product CGST Amount','sgst_amount':'Product SGST Amount','total_cost':'Product Total Amount'})
            
            # total_booth_check_box_list =  {'Product Name','Product Count',' Product GST%','Product CGST Amount','Product SGST Amount','Product Total Amount'}
            # # user_booth_check_box_list = {'Product Name','Product Count'}
            # user_booth_check_box_list = request.data['user_check_box_list']
            # booth_check_box_list = total_booth_check_box_list.difference(user_booth_check_box_list)
            # booth_type_df.drop(booth_check_box_list,inplace= True,axis = 1)
            booth_type_df = booth_type_df.set_index('S.No')

            file_name = str(datetime.datetime.now().date()) + '_Booth_Report.xlsx'
            file_path = os.path.join('static/media/by_product/daily_sale/', file_name)
                
            booth_type_df.to_excel(file_path)
            # print(file_name)

            data['file_name'] = file_name
            try:
                image_path = file_path
                with open(image_path, 'rb') as image_file:
                    encoded_image = b64encode(image_file.read())
                    data['excel'] = encoded_image
            except Exception as err:
                print(err)
            return Response(data=data, status=status.HTTP_200_OK)
                
        elif request.data['report_option'] == "Business Type Wise":
            business_type_ids = request.data['business_type_id']
            # business_type_ids  = [1]
            
            by_sale_group_business_obj=BySaleGroup.objects.filter(ordered_date__range=[data["from_date"],data["to_date"]],business__business_type__in=business_type_ids)
            by_sale_group_business_list=by_sale_group_business_obj.values_list("id","business__code","business__business_type__name","bysale__by_product__quantity","ordered_date","bysale__by_product__name","bysale__count","bysale__by_product__cgst_amount","bysale__by_product__sgst_amount","bysale__total_cost","bysale__by_product__cgst_percent","bysale__by_product__sgst_percent")
            by_sale_group_business_column=["id","business_code","business_type","quantity","ordered_date","by_product_name","bysale_count","cgst_amount","sgst_amount","total_cost","cgst_percent","sgst_percent"]
            by_sale_group_business_df=pd.DataFrame(by_sale_group_business_list,columns=by_sale_group_business_column)
            
            by_sale_group_business_df['GST%'] = by_sale_group_business_df['cgst_percent']+by_sale_group_business_df['sgst_percent']
            by_sale_group_business_df.drop('cgst_percent', inplace=True, axis=1)
            by_sale_group_business_df.drop('sgst_percent', inplace=True, axis=1)
            by_sale_group_business_df.drop('id', inplace=True, axis=1)
            
            agent_business_obj=BusinessAgentMap.objects.filter(business__business_type_id__in=business_type_ids).values_list('agent__first_name',"business__business_type__name")
            agent_business_column=['agent_name',"business_type"]
            agent_business_df=pd.DataFrame(agent_business_obj,columns=agent_business_column)
            
            business_type_df = pd.merge(by_sale_group_business_df, agent_business_df, left_on='business_type', right_on='business_type', how='left')
            
            business_type_df.insert(0, 'S.No', range(1, 1 + len(business_type_df)))
            
            business_type_df = business_type_df.reindex(columns=[
            "S.No","business_code",'agent_name',"business_type","quantity","ordered_date","by_product_name",
            'GST%',"bysale_count","cgst_amount","sgst_amount","total_cost"])
        
            
            business_type_df = business_type_df.rename(columns={'S.No': 'S.No', 'business_code': 'Booth No','agent_name':'Agent Name','business_type':'Business Type','quantity' :'Product Quantity','ordered_date': 'Date','by_product_name':'Product Name','bysale_count':'Product Count','GST%':' Product GST%','cgst_amount':'Product CGST Amount','sgst_amount':'Product SGST Amount','total_cost':'Product Total Amount'})
            
            # total_business_check_box_list =  {'Product Name','Product Count',' Product GST%','Product CGST Amount','Product SGST Amount','Product Total Amount'}
            # # user_business_check_box_list = {'Product Name','Product Count'}
            # user_business_check_box_list = request.data['user_check_box_list']
            # business_check_box_list = total_business_check_box_list.difference(user_business_check_box_list)
            # business_type_df.drop(business_check_box_list,inplace= True,axis = 1)
            business_type_df = business_type_df.set_index('S.No')

            file_name = str(datetime.datetime.now().date()) + '_Business_Report.xlsx'
            file_path = os.path.join('static/media/by_product/daily_sale/', file_name)
                
            business_type_df.to_excel(file_path)
            # print(file_name)

            data['file_name'] = file_name
            try:
                image_path = file_path
                with open(image_path, 'rb') as image_file:
                    encoded_image = b64encode(image_file.read())
                    data['excel'] = encoded_image
            except Exception as err:
                print(err)
            return Response(data=data, status=status.HTTP_200_OK)
            
            
        elif request.data['report_option'] == "Product Wise":
            # by_product_ids = [533,534]
            by_product_ids= request.data['by_product_ids']
            # if by_product_ids == None:
            #     by_product_id = list(ByProduct.objects.filter().values_list('id',flat=True))
            #     print("check")
            by_sale_group_product_obj=BySaleGroup.objects.filter(ordered_date__range=[data["from_date"],data["to_date"]],bysale__by_product_id__in=by_product_ids)
            by_sale_group_product_list=by_sale_group_product_obj.values_list("id","bysale__count","bysale__by_product__cgst_amount","bysale__by_product__quantity","ordered_date","bysale__by_product__sgst_amount","bysale__total_cost","bysale__by_product__cgst_percent","bysale__by_product__sgst_percent","bysale__by_product__name")
            by_sale_group_product_column=["id","bysale_count","cgst_amount","quantity","ordered_date","sgst_amount","total_cost","cgst_percent","sgst_percent","by_product_name"]
            by_sale_group_product_df=pd.DataFrame(by_sale_group_product_list,columns=by_sale_group_product_column)

            by_sale_group_product_df['GST%'] = by_sale_group_product_df['cgst_percent']+by_sale_group_product_df['sgst_percent']
            by_sale_group_product_df.drop('cgst_percent', inplace=True, axis=1)
            by_sale_group_product_df.drop('sgst_percent', inplace=True, axis=1)
            by_sale_group_product_df.drop('id', inplace=True, axis=1)
            
            by_sale_group_product_df.insert(0, 'S.No', range(1, 1 + len(by_sale_group_product_df)))
            
            by_sale_group_product_df = by_sale_group_product_df.reindex(columns=[
            "S.No","by_product_name","quantity","ordered_date",
            'GST%',"bysale_count","cgst_amount","sgst_amount","total_cost"])
            
            by_sale_group_product_df = by_sale_group_product_df.rename(columns={'S.No': 'S.No', 'by_product_name':'Product Name','quantity' :'Product Quantity','ordered_date': 'Date','bysale_count':'Product Count','GST%':' Product GST%','cgst_amount':'Product CGST Amount','sgst_amount':'Product SGST Amount','total_cost':'Product Total Amount'})
            
            # total_product_check_box_list =  {'Product Count',' Product GST%','Product CGST Amount','Product SGST Amount','Product Total Amount'}
            # # user_product_check_box_list = {'Product CGST Amount','Product SGST Amount','Product Total Amount'}
            # user_product_check_box_list = request.data['user_check_box_list']
            # product_check_box_list = total_product_check_box_list.difference(user_product_check_box_list)
            # by_sale_group_product_df.drop(product_check_box_list,inplace= True,axis = 1)
            by_sale_group_product_df = by_sale_group_product_df.set_index('S.No')

            file_name = str(datetime.datetime.now().date()) + '_Product_Report.xlsx'
            file_path = os.path.join('static/media/by_product/daily_sale/', file_name)
                
            by_sale_group_product_df.to_excel(file_path)
            # print(file_name)

            data['file_name'] = file_name
            try:
                image_path = file_path
                with open(image_path, 'rb') as image_file:
                    encoded_image = b64encode(image_file.read())
                    data['excel'] = encoded_image
            except Exception as err:
                print(err)
            return Response(data=data, status=status.HTTP_200_OK)
                
    if data['type'] == "Received":
        if data['report_option'] == "Total GRN":
            # grn_number_ids = ["GRN2200050"]
            grn_number_ids = request.data['grn_number_id']
            goods_receipt_grn_obj = GoodsReceiptRecord.objects.filter(goods_receipt_master__grn_date__range=[data["from_date"],data["to_date"]],goods_receipt_master__grn_number__in=grn_number_ids)
            goods_receipt_grn_list = goods_receipt_grn_obj.values_list('goods_receipt_master__grn_number','goods_receipt_master__purchase_company__company_name','goods_receipt_master__bill_number','goods_receipt_master__bill_date','goods_receipt_master__grn_date','by_product__short_name','by_product__quantity','by_product__bysale__count',
                                                                    'by_product__cgst_percent','by_product__sgst_percent','by_product__cgst_amount','by_product__sgst_amount','by_product__bysale__total_cost')
            goods_receipt_grn_column = ['grn_number','purchase_company','bill_number','bill_date','date','by_product_name','quantity','count','cgst_percent','sgst_percent','cgst_amount','sgst_amount','total_cost']
            goods_receipt_grn_df = pd.DataFrame(goods_receipt_grn_list,columns=goods_receipt_grn_column)
            
            goods_receipt_grn_df['GST%'] = goods_receipt_grn_df['cgst_percent']+goods_receipt_grn_df['sgst_percent']
            goods_receipt_grn_df.drop('cgst_percent', inplace=True, axis=1)
            goods_receipt_grn_df.drop('sgst_percent', inplace=True, axis=1)
            
            goods_receipt_grn_df.insert(0, 'S.No', range(1, 1 + len(goods_receipt_grn_df)))
            goods_receipt_grn_df = goods_receipt_grn_df.reindex(columns=[
            'S.No','grn_number','purchase_company','bill_number','bill_date','date','by_product_name','quantity','count','GST%','cgst_amount','sgst_amount','total_cost'])
            
            
            goods_receipt_grn_df = goods_receipt_grn_df.rename(columns={'S.No': 'S.No','grn_number':'GRN No','purchase_company':'Purchase Company','bill_number':'Bill Number','bill_date':'Bill Date','date':'Date','by_product_name':'Product Name','quantity' :'Product Quantity','count':'Product Count','GST%':'Product GST%','cgst_amount':'Product CGST Amount','sgst_amount':'Product SGST Amount','total_cost':'Product Total Amount'})
            
            goods_receipt_grn_df['Date'] = goods_receipt_grn_df['Date'].apply(lambda a: pd.to_datetime(a).date()) 
            goods_receipt_grn_df['Bill Date'] = goods_receipt_grn_df['Bill Date'].apply(lambda a: pd.to_datetime(a).date())
            goods_receipt_grn_df = goods_receipt_grn_df.set_index('S.No')

            file_name = str(datetime.datetime.now().date()) + '_GRN_Report.xlsx'
            file_path = os.path.join('static/media/by_product/daily_sale/', file_name)
            
            goods_receipt_grn_df.to_excel(file_path)
            #print(file_name)

            data['file_name'] = file_name
            try:
                image_path = file_path
                with open(image_path, 'rb') as image_file:
                    encoded_image = b64encode(image_file.read())
                    data['excel'] = encoded_image
            except Exception as err:
                print(err)
            return Response(data=data, status=status.HTTP_200_OK)
            
        elif data['report_option'] == "Product Wise":
            # by_product_ids = [533]
            by_product_ids = request.data['by_product_ids']
            goods_receipt_product_obj = GoodsReceiptRecord.objects.filter(goods_receipt_master__grn_date__range=[data["from_date"],data["to_date"]],by_product_id__in=by_product_ids)
            goods_receipt_product_list = goods_receipt_product_obj.values_list('goods_receipt_master__grn_number','goods_receipt_master__purchase_company__company_name','goods_receipt_master__bill_number','goods_receipt_master__bill_date','goods_receipt_master__grn_date','by_product__short_name','by_product__quantity','by_product__bysale__count',
                                                                    'by_product__cgst_percent','by_product__sgst_percent','by_product__cgst_amount','by_product__sgst_amount','by_product__bysale__total_cost')
            goods_receipt_product_column = ['grn_number','purchase_company','bill_number','bill_date','date','by_product_name','quantity','count','cgst_percent','sgst_percent','cgst_amount','sgst_amount','total_cost']
            goods_receipt_product_df = pd.DataFrame(goods_receipt_product_list,columns=goods_receipt_product_column)
            
            goods_receipt_product_df['GST%'] = goods_receipt_product_df['cgst_percent']+goods_receipt_product_df['sgst_percent']
            goods_receipt_product_df.drop('cgst_percent', inplace=True, axis=1)
            goods_receipt_product_df.drop('sgst_percent', inplace=True, axis=1)
            
            goods_receipt_product_df.insert(0, 'S.No', range(1, 1 + len(goods_receipt_product_df)))
            goods_receipt_product_df = goods_receipt_product_df.reindex(columns=[
            'S.No','by_product_name','grn_number','purchase_company','bill_number','bill_date','date','quantity','count','GST%','cgst_amount','sgst_amount','total_cost'])
            
            
            goods_receipt_product_df = goods_receipt_product_df.rename(columns={'S.No': 'S.No','by_product_name':'Product Name','grn_number':'GRN No','purchase_company':'Purchase Company','bill_number':'Bill Number','bill_date':'Bill Date','date':'GRN Date','quantity' :'Product Quantity','count':'Product Count','GST%':'Product GST%','cgst_amount':'Product CGST Amount','sgst_amount':'Product SGST Amount','total_cost':'Product Total Amount'})
            
            goods_receipt_product_df['GRN Date'] = goods_receipt_product_df['GRN Date'].apply(lambda a: pd.to_datetime(a).date()) 
            goods_receipt_product_df['Bill Date'] = goods_receipt_product_df['Bill Date'].apply(lambda a: pd.to_datetime(a).date())
            
            # total_grn_product_check_box_list =  {'Product Name'}
            # # user_grn_product_check_box_list = {'Product CGST Amount','Product SGST Amount','Product Total Amount'}
            # user_grn_product_check_box_list = request.data['user_check_box_list']
            # grn_product_check_box_list = total_grn_product_check_box_list .difference(user_grn_product_check_box_list)
            # goods_receipt_product_df.drop(grn_product_check_box_list,inplace= True,axis = 1)
            goods_receipt_product_df = goods_receipt_product_df.set_index('S.No')

            file_name = str(datetime.datetime.now().date()) + '_GRN_Product_Report.xlsx'
            file_path = os.path.join('static/media/by_product/daily_sale/', file_name)
            
            goods_receipt_product_df.to_excel(file_path)
            # print(file_name)

            data['file_name'] = file_name
            try:
                image_path = file_path
                with open(image_path, 'rb') as image_file:
                    encoded_image = b64encode(image_file.read())
                    data['excel'] = encoded_image
            except Exception as err:
                print(err)
            return Response(data=data, status=status.HTTP_200_OK)
            
        elif data['report_option'] == "Purchase Company":
            # purchase_company_ids = [2]
            purchase_company_ids = request.data['purchase_company_id']
            #  goods_receipt_obj=GoodsReceiptRecord.objects.filter(goods_receipt_master__grn_date__range=[data["from_date"],data["to_date"]],goods_receipt_code=goods_receipt_code,goods_receipt_master__purchase_company__company_name=purchase_company_list,by_product__short_name=by_product_list)
            goods_receipt_purchase_obj = GoodsReceiptRecord.objects.filter(goods_receipt_master__grn_date__range=[data["from_date"],data["to_date"]],goods_receipt_master__purchase_company__in=purchase_company_ids)
            goods_receipt_purchase_list = goods_receipt_purchase_obj.values_list('goods_receipt_master__grn_number','goods_receipt_master__purchase_company__company_name','goods_receipt_master__bill_number','goods_receipt_master__bill_date','goods_receipt_master__grn_date','by_product__short_name','by_product__quantity','by_product__bysale__count',
                                                                    'by_product__cgst_percent','by_product__sgst_percent','by_product__cgst_amount','by_product__sgst_amount','by_product__bysale__total_cost')
            goods_receipt_purchase_column = ['grn_number','purchase_company','bill_number','bill_date','date','by_product_name','quantity','count','cgst_percent','sgst_percent','cgst_amount','sgst_amount','total_cost']
            goods_receipt_purchase_df = pd.DataFrame(goods_receipt_purchase_list,columns=goods_receipt_purchase_column)
            
            goods_receipt_purchase_df['GST%'] = goods_receipt_purchase_df['cgst_percent']+goods_receipt_purchase_df['sgst_percent']
            goods_receipt_purchase_df.drop('cgst_percent', inplace=True, axis=1)
            goods_receipt_purchase_df.drop('sgst_percent', inplace=True, axis=1)
            
            goods_receipt_purchase_df.insert(0, 'S.No', range(1, 1 + len(goods_receipt_purchase_df)))
            goods_receipt_purchase_df = goods_receipt_purchase_df.reindex(columns=[
            'S.No','grn_number','purchase_company','bill_number','bill_date','date','by_product_name','quantity','count','GST%','cgst_amount','sgst_amount','total_cost'])
            
            
            goods_receipt_purchase_df = goods_receipt_purchase_df.rename(columns={'S.No': 'S.No','grn_number':'GRN No','purchase_company':'Purchase Company','bill_number':'Bill Number','bill_date':'Bill Date','date':'GRN Date','by_product_name':'Product Name','quantity' :'Product Quantity','count':'Product Count','GST%':'Product GST%','cgst_amount':'Product CGST Amount','sgst_amount':'Product SGST Amount','total_cost':'Product Total Amount'})
                
            goods_receipt_purchase_df['GRN Date'] = goods_receipt_purchase_df['GRN Date'].apply(lambda a: pd.to_datetime(a).date()) 
            goods_receipt_purchase_df['Bill Date'] = goods_receipt_purchase_df['Bill Date'].apply(lambda a: pd.to_datetime(a).date())    
                
            # total_purchase_company_check_box_list =  {'Purchase Company'}
            # # user_purchase_company_check_box_list = {'Product CGST Amount','Product SGST Amount','Product Total Amount'}
            # user_purchase_company_check_box_list = request.data['user_check_box_list']
            # purchase_company_check_box_list = total_purchase_company_check_box_list.difference(user_purchase_company_check_box_list)
            # goods_receipt_purchase_df.drop(purchase_company_check_box_list,inplace= True,axis = 1)
            goods_receipt_purchase_df=goods_receipt_purchase_df.set_index('S.No')
            
            file_name = str(datetime.datetime.now().date()) + '_Purchase_Report.xlsx'
            file_path = os.path.join('static/media/by_product/daily_sale/', file_name)
            
            goods_receipt_purchase_df.to_excel(file_path)
            
            # print(file_name)

            data['file_name'] = file_name
            try:
                image_path = file_path
                with open(image_path, 'rb') as image_file:
                    encoded_image = b64encode(image_file.read())
                    data['excel'] = encoded_image
            except Exception as err:
                print(err)
            return Response(data=data, status=status.HTTP_200_OK)
                

In [34]:
def serve_qty_and_short_name(start_date, end_date):
    product_dict = {}
    for product in Product.objects.filter():
        if not product.id in product_dict:
            product_dict[product.id] = {}
        trace_ids = []
        for product_trace in ProductNameAndQuantityTrace.objects.filter(product_id=product.id):
            date_list = pd.date_range(product_trace.start_date, product_trace.end_date)
            if start_date in date_list :
                trace_ids.append(product_trace.id)
            if end_date in date_list:
                trace_ids.append(product_trace.id)  
        if len(set(trace_ids)) == 0:
            continue     
        if len(set(trace_ids)) == 2:
            continue  
        if len(set(trace_ids)) == 1:
            product_trace_id = trace_ids[0]
            product_trace_obj = ProductNameAndQuantityTrace.objects.get(id=product_trace_id)
            data_dict = {
                'qty': product_trace_obj.quantity,
                'short_name': product_trace_obj.short_name
            }
            product_dict[product.id] = data_dict
        else:
            data_dict = {
                'is_multiple_qty': True,
            }
            return data_dict
    final_dict = {
        'is_multiple_qty' : False,
        'product_dict': product_dict
    }
    return final_dict


In [35]:
def find_base_price_from_total_cost(total_cost, total_gst):
    whole_gst = (total_gst / 100) + 1
    base_price = Decimal(total_cost) / whole_gst
    return round(base_price, 2)


In [38]:
# def serve_govt_jst_bill_summary(request):
# print(request.data)
# selected_business_type_ids = request.data['selected_business_type_id']
# start_date = request.data['start_date']
# end_date = request.data['end_date']


import pandas as pd
import datetime
from decimal import *
from base64 import b64encode, b64decode

selected_business_type_ids = [1,2,3]
start_date = "2022-09-01"
end_date = "2022-09-15"
business_codes = list(Business.objects.filter(business_type_id__in=selected_business_type_ids).values_list('code', flat=True))

business_wise_bill = BusinessWiseBillNumber.objects.filter(business__code__in=business_codes, from_date=start_date, to_date=end_date,product_group_type=2)
business_wise_bill_list = list(business_wise_bill.values_list('business_id', 'bill_number'))
business_wise_bill_columns = ['business_id', 'bill_number']
business_wise_bill_df = pd.DataFrame(business_wise_bill_list, columns=business_wise_bill_columns)
print('business_wise_bill_df',business_wise_bill_df)
# date_in_format = datetime.strptime(str(end_date), '%Y-%m-%d')
# date = datetime.strftime(date_in_format, '%d-%b-%Y')
date_in_format = "2022-09-11"
date="2022-09-11"
product_quantity_check_dict = serve_qty_and_short_name(start_date, end_date)
sale_obj = Sale.objects.filter(sale_group__business__code__in=business_codes, sale_group__date__gte=start_date, sale_group__date__lte=end_date, product__group_id=2).exclude(product_id=28)
sale_list = list(sale_obj.values_list('id', 'sale_group__business__user_profile_id','sale_group__business_id', 'sale_group__business__code', 'product_id', 'count', 'product__cgst_percent', 'product__sgst_percent', 'cost', 'product__hsn_code', 'sale_group__business__businessagentmap__agent__first_name', 'sale_group__business__businessagentmap__agent__agent_profile__gst_number'))
sale_column = ['id', 'user_profile_id', 'business_id', 'business_code', 'product_id', 'count', 'cgst_percent', 'sgst_percent' ,'cost', 'hsn_code', 'agent_name', 'gst_number']
sale_df = pd.DataFrame(sale_list, columns=sale_column)

sale_df = sale_df.groupby(['business_id', 'product_id']).agg({'count': sum, 'user_profile_id': 'first', 'business_code': 'first', 'cgst_percent': 'first', 'sgst_percent': 'first', 'cost': sum, 'hsn_code': 'first', 'agent_name': 'first', 'gst_number': 'first'}).reset_index()
short_name_and_qty = product_quantity_check_dict['product_dict']

if sale_df.empty:
    document = {}
    document['is_data_available'] = False
#     return Response(data=document, status=status.HTTP_200_OK)
sale_df['cost'] = sale_df['cost'].astype(float)
sale_df['mrp'] = sale_df['cost'] / sale_df['count']
sale_df['product_quantity'] = sale_df.apply(lambda x: short_name_and_qty[x['product_id']]['qty'], axis=1)
sale_df['product_short_name'] = sale_df.apply(lambda x: short_name_and_qty[x['product_id']]['short_name'], axis=1)
sale_df['cost_by_base_price'] = sale_df.apply(lambda x: find_base_price_from_total_cost(x['cost'], x['cgst_percent'] + x['sgst_percent']), axis=1)
sale_df['cost_by_base_price'] = sale_df['cost_by_base_price'].astype(float)
sale_df['base_price'] = sale_df['cost_by_base_price'] / sale_df['count']
sale_df['product_quantity'] = sale_df['product_quantity'].astype(float)
sale_df['total_gst_for_base_price'] = sale_df['cost'] - sale_df['cost_by_base_price']
sale_df['qty_in_whole'] = (sale_df['count'] * sale_df['product_quantity']) / 1000
sale_df['total_gst'] = sale_df['cgst_percent'] + sale_df['sgst_percent']
sale_df['total_cost'] = (sale_df['cost_by_base_price'] + sale_df['total_gst_for_base_price'])

sale_df = sale_df.merge(business_wise_bill_df, how="left", on="business_id")
sale_df = sale_df[sale_df['bill_number'] != ""]
sale_df = sale_df.fillna("")
# sheet 1
grouped_sale_df = sale_df.groupby(['business_id', 'total_gst']).agg({'agent_name': 'first','gst_number': 'first','bill_number': 'first', 'total_cost': 'sum','cost_by_base_price': sum , 'total_gst_for_base_price': sum}).reset_index()
grouped_sale_df['total_cost_rounded'] = grouped_sale_df['total_cost'].astype(float)
grouped_sale_df['cgst_value'] = grouped_sale_df['total_gst_for_base_price'] / 2
grouped_sale_df['sgst_value'] = grouped_sale_df['total_gst_for_base_price'] / 2
grouped_sale_df['bill_date'] = date

final_df = grouped_sale_df.rename(columns = {'gst_number': 'GSTIN_UINOFRECIPIENT', 'agent_name': 'NAME', 'bill_number': 'INVOICENO', 'bill_date' : 'INVOICEDATE', 'total_cost_rounded': 'INVOICEVALUE', 'total_gst': 'RATE', 'cost_by_base_price': 'TAXABLEVALUE', 'cgst_value': 'CGST', 'sgst_value': 'SGST'})
final_df['PLACEOFSUPPLY'] = ''
final_df['REVERSECHARGE'] = ''
final_df['INVOICETYPE'] = ''
final_df['IGST'] = 0
final_df = final_df[['GSTIN_UINOFRECIPIENT', 'NAME', 'INVOICENO', 'INVOICEDATE', 'INVOICEVALUE', 'PLACEOFSUPPLY', 'REVERSECHARGE', 'INVOICETYPE', 'RATE', 'TAXABLEVALUE', 'CGST', 'SGST', 'IGST']]

# sheet 2
product_wise_df = sale_df
product_wise_df['bill_date'] = date
product_wise_df['cgst_value'] = product_wise_df['total_gst_for_base_price'] / 2
product_wise_df['sgst_value'] = product_wise_df['total_gst_for_base_price'] / 2
product_wise_df['ISGT'] = 0.000
product_wise_df = product_wise_df.rename(columns={'agent_name': 'AGENT', 'gst_number': 'BTAXNO', 'product_short_name': 'PRODUCT', 'bill_number': 'INVNO', 'bill_date': 'INVDATE', 'sgst_value': 'SGST','cgst_value': 'CGST', 'total_cost': 'NETAMOUNT', 'hsn_code': 'HSNCODE', 'total_gst': 'GST', 'mrp': 'RATE'})
final_df_2 = product_wise_df[['AGENT', 'BTAXNO', 'PRODUCT', 'INVNO', 'INVDATE', 'RATE', 'GST', 'SGST', 'CGST', 'ISGT','NETAMOUNT', 'HSNCODE']]

file_name = f"static/media/monthly_report/{start_date} - {end_date} - GVT Fermented GST Bill Summary.xlsx"
writer = pd.ExcelWriter(file_name, engine="xlsxwriter")
sheet_name1 = "sheet1"
sheet_name2 = "sheet2"
final_df.to_excel(writer, sheet_name=sheet_name1, index=False)
final_df_2.to_excel(writer, sheet_name=sheet_name2, index=False)
writer.save()
document = {}
document['is_data_available'] = True
document['file_name'] = f"{start_date} - {end_date} - GVT Fermented GST Bill Summary.xlsx"
try:
    image_path = file_name
    with open(image_path, 'rb') as image_file:
        encoded_image = b64encode(image_file.read())
        document['excel'] = encoded_image
except Exception as err:
    print(err)
    document = {}
# return Response(data=document, status=status.HTTP_200_OK)



business_wise_bill_df Empty DataFrame
Columns: [business_id, bill_number]
Index: []


In [ ]:
x_60=[12,102,147,165,210,219,120,57,66,75,84,93,129,174,183,201,228,489,399,408,498,417,426,435,453,462,471,480,579,615,633,597,768,849,867,885,1020,1029,858,876,1038,1065,1074,1083,1047,1398,1407,1461,1470,1506,1515,1524,1488,1452,1704,1713,1740,1749,1731,1695,1875,1884,1893,1902,1911,1938,1857,2037,48,588,606,1920,894,1092,1479,1929,1722,624,1389,156,192,1056,2273,2213,2456,111,138,1497,2629,2643,2650,2657,2664,2671,2678,2699,2706,2713,2720,2727,2734,2741,2685,2776,2783,2790,2797,2804,2811,2636,2692,2748,2755,2762,2769,2818
]

In [2]:
x=BusinessWiseProductDiscountTrace.objects.filter(mrp=48,end_date='2051-05-01 05:30:00.000').update(end_date='2022-11-04 05:30:00.000')
x

0

In [1]:
customersale_group_delete = ICustomerSaleGroup.objects.filter(id=283998).delete()
customersale_group_delete

(4,
 {'main.ICustomerMonthlyOrderTransaction_icustomer_sale_group': 1,
  'main.CounterEmployeeTraceSaleGroupMap_icustomer_sale_group': 1,
  'main.ICustomerSaleGroupPaymentRequestMap_sale_group': 1,
  'main.ICustomerWalletAmoutForOrder_icustomer_sale_group': 0,
  'main.ICustomerSaleGroup': 1})

In [1]:
for i in BusinessWiseDailySaleUpdate.objects.filter():
    i.sales_quantity = i.received_quantity
    print(i.received_quantity)
    i.save()

30
40
50
20
60
27
20
5
1
10
20
20
5
5
5
10
5
3
5
10
5
5
1
5
10
10
10
24
200
250
50
20
40
50
120
10
10
30
5
40
5
75
20
1
100
100
50
25
20
20
160
40
30
24
50
15
50
20
5
10
10
10
5
81
10
15
10
5
10
8
20
20
5
25
25
10
20
10
5
30
10
3
20
20
20
12
81
30
3
40
50
40
60
5
50
30
60
120
270
20
20
50
120
135
54
90
50
20
120
115
25
20
20
15
5
2
3
2
15
5
2
2
2
2
40
80
36
100
40
60
18
36
20
50
30
18
18
40
4
30
210
20
40
20
108
20
1
20
15
15
10
40
15
20
3
3
30
1
100
36
6
240
5
2
40
10
20
100
15
10
15
5
1000
240
20
30
3
100
40
135
20
120
270
30
20
10
20
20
50
27
150
14
19
30
20
40
30
40
50
60
50
60
180
10
70
5
30
50
135
100
30
10
20
10
40
20
5
120
50
10
10
30
20
5
120
135
10
10
64
15
32
64
15
40
18
50
20
32
15
1
32
18
1
32
18
1
1
40
100
50
128
60
6
32
32
18
3
15
3
2
50
20
20
10
10
50
25
32
2
32
250
15
72
10
72
10
36
72
60
15
80
50
200
100
32
10
32
5
32
2
10
5
10
20
1
5
12
20
10
10
1
20
50
10
100
60
100
10
2
20
10
20
50
40
40
140
100
135
15
20
5
50
30
20
2
30
20
25
25
29
7
5
10
3
10
120
270
5
10
20
4
12

20
100
32
36
20
2
40
50
50
40
18
100
100
30
20
50
2
40
100
100
20
4
20
10
60
20
300
10
100
50
20
20
20
6
30
10
20
50
25
20
60
27
20
3
20
25
15
2
20
25
20
15
24
27
15
7
4
15
10
1
50
30
27
100
20
10
6
50
50
4
20
60
30
20
100
50
5
20
100
1
48
10
10
5
4
60
70
10
5
30
40
50
20
30
20
5
27
5
5
5
5
20
50
40
100
15
25
60
60
100
150
40
36
18
36
18
20
40
40
20
75
100
54
36
3
15
1
25
10
5
25
20
25
20
30
20
3
10
20
10
5
10
10
5
1
10
20
10
10
10
20
10
70
80
40
6
6
30
50
5
50
5
20
40
5
50
20
20
20
30
100
20
20
50
50
50
50
10
50
50
20
50
50
10
65
20
10
10
40
20
20
3
10
5
5
3
3
5
5
10
10
20
10
10
15
10
10
5000
10
10
10
1500
750
20
18
20
10
40
50
40
25
25
36
11
50
30
25
2
2
10
10
110
30
5
20
10
50
5
100
20
5
40
50
50
10
10
2
20
50
50
15
5
20
20
60
10
20
15
50
20
30
100
20
10
10
10
30
20
10
10
5
50
20
10
10
5
20
20
50
100
150
120
135
10
10
10
5
10
20
85
3
150
50
100
150
270
20
162
24
4
10
20
40
246
50
10
5
2
10
10
10
12
10
5
10
20
20
20
15
120
17
10
10
20
30
10
5
5
10
20
3
20
10
2
10
20
50
20
20
3
10
20


5
4
1
1
4
50
30
5
20
20
10
10
10
1
1
4
20
30
5
10
20
10
10
10
1
1
4
110
180
5
40
20
14
10
6
6
4
20
25
60
50
30
10
20
2
2
5
25
2
20
2
30
10
25
20
10
5
10
10
50
50
40
20
100
30
20
60
20
20
120
30
10
20
50
20
60
10
50
50
40
2
120
10
50
50
100
100
5
100
10
240
14
20
25
10
1
4
30
10
10
10
10
20
24
30
10
2
5
20
20
10
20
24
2
15
3
10
12
20
5
2
10
20
10
36
15
5
10
20
40
5
40
20
30
20
30
5
50
50
20
30
5
100
100
100
5
40
25
20
2
2
12
10
25
10
50
50
120
250
20
20
20
5
2
20
20
5
5
36
20
10
32
32
32
18
18
75
40
3
3
3
2
2
2
2
2
2
40
100
54
18
18
18
60
60
10
5
5
4
1
20
1
20
50
50
2
2
36
36
36
20
25
2
2
40
100
30
10
2
30
50
50
20
10
2
40
20
6
10
10
20
5
5
5
20
5
10
3
30
10
5
6
20
10
20
20
100
200
5
100
60
100
240
19
20
15
2
10
20
15
10
12
5
2
3
5
10
12
3
20
100
20
10
10
10
10
10
1
30
50
5
10
3
2
24
150
10
22
40
50
10
50
4
30
20
50
5
30
105
150
5
20
40
120
60
15
10
10
10
50
3
10
48
7
10
50
80
20
50
40
40
40
96
18
18
3
3
3
3
3
3
1
50
20
20
18
18
18
2
2
2
1
1
36
54
5
10
10
125
60
40
5
5
100
125
64
64
32


3
3
20
50
10
10
70
100
20
10
5
18
200
150
300
600
30
100
108
945
660
64
32
18
18
36
100
20
40
100
60
50
60
20
20
20
50
100
60
3
3
3
20
75
60
20
20
20
80
10
100
100
60
5
5
5
60
50
10
3
3
10
20
5
20
20
60
20
60
36
18
18
4
5
14
5
5
20
20
20
20
18
18
32
32
32
3
3
3
3
3
3
15
5
3
5
5
20
60
10
10
20
1
10
2
2
40
50
20
4
4
4
50
60
5
5
50
75
10
20
40
120
30
50
10
50
20
5
30
30
5
20
40
120
200
200
12
240
60
60
3
20
8
30
20
20
108
250
20
20
20
25
10
8
5
10
20
2000
400
80
80
4
2
10
10
3
5
80
200
80
100
3
20
20
20
10
5
3
10
80
250
40
10
20
5
1
15
5
10
25
25
14
200
250
20
100
5
5
5
5
5
5
50
20
15
40
200
6348
10
50
40
20
10
5
5
5
5
5
5
200
20
4
50
50
20
3
30
20
2
50
50
30
50
60
10
3
10
10
20
24
27
10
20
20
10
4
1
6
30
4
10
10
20
2
50
200
100
25
20
24
5
30
30
10
2
5
20
50
20
36
54
2
2
10
20
2
5
27
10
10
5
10
120
20
20
25
10
500
40
10
3
100
40
10
10
250
80
50
50
40
24
20
20
25
10
25
20
10
10
20
25
1
15
30
25
1
20
50
5
10
1
4
20
50
20
12
5
8
30
10
40
50
20
30
40
100
20
30
40
50
50
10
40
30
20
50
25
20
5


6
4
4
4
2
2
4
4
4
4
4
16
4
8
4
4
42
7
1
2
8
4
1
10
2
4
6
6
4
4
34
4
7
2
11
8
4
4
34
10
4
40
5
8
2
2
40
8
48
46
12
4
1
8
132
10
84
8
32
5
16
8
4
2
4
10
1
2
40
29
20
50
3
4
20
20
12
4
4
4
4
1
4
12
6
5
7
4
2
2
8
4
4
4
2
2
24
10
10
6
4
4
4
8
40
38
12
4
4
12
5
24
2
13
17
20
6
2
6
12
4
8
12
4
10
1
2
4
40
8
5
2
4
4
44
6
2
5
8
24
20
8
2
18
5
20
20
12
10
20
20
25
80
1
20
12
16
8
6
16
4
5
4
4
5
32
5
4
20
20
120
80
100
12
5
12
80
2
5
4
12
2
28
24
12
16
20
4
10
44
4
5
4
60
142
25
40
160
54
50
5
20
30
50
189
30
10
50
84
54
10
10
12
5
10
10
10
10
50
36
10
10
10
50
135
10
40
10
50
72
30
30
270
50
50
40
50
30
50
30
50
135
40
30
10
30
30
5
200
100
50
27
100
54
20
10
10
90
50
3
3
40
20
10
20
5
6
3
40
10
40
20
14
15
4
4
2
2
30
25
5
45
28
14
20
20
22
2
10
40
3
50
2
2
1
49
4
5
40
16
40
49
25
51
15
31
50
5
5
20
4
8
20
40
8
20
10
15
6
12
20
10
10
20
20
40
2
45
20
10
20
11
5
75
23
2
20
40
80
40
20
5
10
20
100
20
38
75
20
45
12
20
80
30
20
10
5
50
5
5
20
5
10
12
8
30
30
10
40
50
120
40
100
45
10
20
10
5
5
5
4


In [ ]:
TEAMATE500 --- NEW PRODUCT

In [1]:
import datetime
current_date = datetime.datetime.now()
end_date = "2070-01-05"
quantity_obj = ProductNameAndQuantityTrace.objects.create(start_date = current_date,end_date = end_date,short_name = 'Tea 500', quantity = 500, product_id = 33, unit_id = 1)
quantity_obj

<ProductNameAndQuantityTrace: ProductNameAndQuantityTrace object>

In [ ]:
subgroup_obj = ProductSubGroup.objects.create(name = 'TeaMate')

In [4]:
ProductProductSubGroupMap_obj = ProductProductSubGroupMap.objects.create(product_id = 33, product_sub_group_id = 5)

In [6]:
ProductTrayConfig_obj = ProductTrayConfig.objects.create(product_id = 33,tray_count = 1,product_count = 24,created_by_id = 1, modified_by_id = 1)

In [7]:
business_type_id = [1,2,3,16]
for i in business_type_id:
    BusinessProductConcessionMap.objects.create(business_type_id=i,concession_type_id=1,product_id=33)

In [8]:
business_type_id = [4,5,6,7,8,10,12,15]
for i in business_type_id:
    BusinessProductConcessionMap.objects.create(business_type_id=i,concession_type_id=2,product_id=33)

In [9]:
business_type_id = [1,2]
for i in business_type_id:
    BusinessTypeWiseProductCommission.objects.create(product_id =33,business_type_id = i,commission_percentage = 4.1,created_by_id =1,modified_by_id = 1)

In [11]:
BusinessTypeWiseProductCommission.objects.create(product_id =33,business_type_id = 3,commission_percentage = 0,created_by_id =1,modified_by_id = 1)

<BusinessTypeWiseProductCommission: BusinessTypeWiseProductCommission object>

In [12]:
customer_type_id = [1,2,3,5]
for i in customer_type_id:
    ICustomerTypeWiseProductDiscount.objects.create(product_id = 33,customer_type_id = i,discounted_price=30,created_by_id = 1, modified_by_id = 1)

In [ ]:
TEAMATE1000 --- NEW PRODUCT

In [1]:
import datetime
current_date = datetime.datetime.now()
end_date = "2070-01-05"
quantity_obj = ProductNameAndQuantityTrace.objects.create(start_date = current_date,end_date = end_date,short_name = 'Tea 1000', quantity = 1000, product_id = 34, unit_id = 1)
quantity_obj

<ProductNameAndQuantityTrace: ProductNameAndQuantityTrace object>

In [2]:
ProductProductSubGroupMap_obj = ProductProductSubGroupMap.objects.create(product_id = 34, product_sub_group_id = 5)

In [3]:
ProductTrayConfig_obj = ProductTrayConfig.objects.create(product_id = 34,tray_count = 1,product_count = 12,created_by_id = 1, modified_by_id = 1)

In [4]:
business_type_id = [1,2,3,16]
for i in business_type_id:
    BusinessProductConcessionMap.objects.create(business_type_id=i,concession_type_id=1,product_id=34)

In [5]:
business_type_id = [4,5,6,7,8,10,12,15]
for i in business_type_id:
    BusinessProductConcessionMap.objects.create(business_type_id=i,concession_type_id=2,product_id=34)

In [6]:
business_type_id = [1,2]
for i in business_type_id:
    BusinessTypeWiseProductCommission.objects.create(product_id =34,business_type_id = i,commission_percentage = 4.1,created_by_id =1,modified_by_id = 1)

In [7]:
BusinessTypeWiseProductCommission.objects.create(product_id =34,business_type_id = 3,commission_percentage = 0,created_by_id =1,modified_by_id = 1)

<BusinessTypeWiseProductCommission: BusinessTypeWiseProductCommission object>

In [8]:
customer_type_id = [1,2,3,5]
for i in customer_type_id:
    ICustomerTypeWiseProductDiscount.objects.create(product_id = 34,customer_type_id = i,discounted_price=60,created_by_id = 1, modified_by_id = 1)

In [19]:
x=list(BusinessWiseProductDiscountTrace.objects.filter(mrp=390 ,business_wise_discount_id__business_id__business_type_id__in=[3], discounted_price=390).values_list('business_wise_discount_id__business_id__business_type_id','id'))
x

[(3, 5736)]

In [8]:
import datetime
date = datetime.datetime.now()
business_type_wise_commission_id = [16,17]
for i in business_type_wise_commission_id:
    obj=BusinessTypeWiseProductCommissionTrace.objects.create(start_date=date,mrp=50,commission_percentage=4.1,business_type_wise_commission_id=i,product_commission_ended_by_id=1,product_commission_started_by_id=1)

/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField BusinessTypeWiseProductCommissionTrace.start_date received a naive datetime (2022-12-13 04:56:19.199152) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [10]:
 obj=BusinessTypeWiseProductCommissionTrace.objects.create(start_date=date,mrp=30,commission_percentage=0,business_type_wise_commission_id=18,product_commission_ended_by_id=1,product_commission_started_by_id=1)

In [9]:
import datetime
date = datetime.datetime.now()
business_type_wise_commission_id = [19,20]
for i in business_type_wise_commission_id:
    obj=BusinessTypeWiseProductCommissionTrace.objects.create(start_date=date,mrp=60,commission_percentage=4.1,business_type_wise_commission_id=i,product_commission_ended_by_id=1,product_commission_started_by_id=1)

/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField BusinessTypeWiseProductCommissionTrace.start_date received a naive datetime (2022-12-13 04:57:40.988451) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [11]:
obj=BusinessTypeWiseProductCommissionTrace.objects.create(start_date=date,mrp=60,commission_percentage=0,business_type_wise_commission_id=21,product_commission_ended_by_id=1,product_commission_started_by_id=1)

In [5]:
import datetime
date = datetime.datetime.now()
date

datetime.datetime(2022, 12, 13, 4, 47, 47, 89159)

In [14]:
business_id_34 = [1442,1527,1524,1529,1530,1536,1440,1537,1513,1541,1544,1546,1521,1525,1528,1535,1538,1539,1540,1542,1543,1549,1552,1553,1556,1554,1501,1560,1561,1753,1558,1559,1564,1566,1571,1572,1573,1557,1766,1745,1746,1747,1748,1749,1750,1751,1752,1754,1759,1762,1763,1765,1770,1744,1769,1771,1772,1773,1774,1767,1861,1947,2013,2024,2028,2014,2015,2016,2018,2019,2020,2021,2026,2025,2032,2027,2031,2011,2039,2040,2041,2044,2046,2047,2050,2052,2053,2054,2055,2058,2034,2153,2133,2134,2143,2144,2145,2146,2147,2148,2150,2151,2152,2154,2402,2155,2156,2141,2335,2399,2400,2403,2411,2417,2418,2423,2405,2409,2410,2412,2413,2414,2419,2429,2431,2420,2421,2424,2425,2426,2427,2430,2432,2404,2525,2544,2546,2547,2529,2530,2531,2532,2533,2534,2535,2543,2549,2568,1520,1548,2550,2526,1545,1514,1758,1768,2029,2036,2149,2422,2548,1755,2142,2542,1575,2596,2747,1547,
2554,2551,2552,2553,2746,2759,2774,2751,2779,2752,2761,2750,2773,2775,2780,2790,2794,1441,1439,1522,2012,2042,2130,2056,3216,2401,2528,1568,2825,3228,3331,3366,3268,3542,3546,3551,3552,3553,3554,3559,3560,3561,3562,2555,3583,1512,1516,1518,1519,1523,1526,1531,1532,1534,1550,1551,1562,1565,1569,1570,1574,1567,1576,1756,1757,1760,1764,2017,2035,2037,2038,2043,2045,2049,2051,2384,2389,2406,2407,2415,2416,2428,2527,2545,1761,2030,2048,1533,3563,3564,3565,3566,3567,3568,3569,2408,2748,2749,2781,2797,2799,2815,2816,2822,1555,1735,2023,2033,3543,3544,3545,3547,3548,3549,3550,3555,3556,3557,3558]

In [15]:
for i in business_id_34:
        BusinessWiseProductDiscount.objects.create(discounted_price=60, business_id=i, created_by_id=1,modified_by_id=1,product_id=34)

In [12]:
business_id_33 = [1512,1516,1518,1519,1523,1526,1524,1529,1530,1536,1439,1440,1442,1441,1541,1544,1546,1532,1534,1547,1550,1537,1522,1525,1528,1535,1538,1539,1540,1542,1543,1549,1552,1553,1554,1521,1560,1561,1565,1501,1569,1570,1574,1567,1576,1556,1753,1756,1757,1760,1764,1559,1564,1566,1572,1573,1557,1735,1558,1746,1747,1748,1749,1750,1751,1752,1754,1759,1763,1766,1745,2017,1770,1744,1769,1771,1772,1773,1774,1767,1861,2013,1765,2014,2024,2025,2028,2015,2016,2018,2019,2020,2026,2023,2012,2033,2035,2037,2038,2043,2045,2049,2051,2031,2011,2039,2032,2027,2041,2042,2044,2046,2047,2050,2052,2053,2054,2055,2058,2034,2040,2153,2134,2130,2143,2144,2145,2146,2147,2148,2150,2151,2152,2154,2133,2402,2403,2384,2407,2389,2406,2156,2141,2335,2399,2401,2155,2415,2416,2417,2418,2423,2428,2410,2412,2413,2414,2419,2411,2409,2527,2421,2424,2425,2426,2427,2430,2420,2432,2431,2525,
2544,2545,2546,2547,2530,2531,2532,2534,2535,2543,2549,2528,2529,1520,1548,1568,1761,2030,2048,2408,2526,1545,1514,2568,2550,1758,1768,2029,2036,2056,2149,2422,2548,1755,2142,2542,1575,2551,2536,2596,2650,2749,2554,2747,1551,1533,1555,2748,2429,2555,2552,2553,2746,2759,2774,2775,2780,2781,2750,2752,2761,2751,2773,2779,2790,2021,2874,2799,2825,2816,2822,2794,2797,2815,1527,1531,1562,1513,1571,1762,1947,2400,2405,2404,2533,3216,3228,3267,3293,3331,3435,3268,3433,3543,3542,3544,3545,3547,3548,3550,3551,3552,3553,3554,3555,3556,3557,3558,3559,3560,3561,3562,3366,3583,3546,3549,3563,3564,3565,3566,3567,3568,3569]

In [13]:
for i in business_id_33:
        BusinessWiseProductDiscount.objects.create(discounted_price=30, business_id=i, created_by_id=1,modified_by_id=1,product_id=33)

In [17]:
product_33=list(BusinessWiseProductDiscount.objects.filter(discounted_price=30,product_id=33).values_list('id',flat=True))

In [18]:
import datetime
date = datetime.datetime.now()
for i in product_33:
    BusinessWiseProductDiscountTrace.objects.create(start_date=date,end_date='2051-05-01',mrp=30,discounted_price=30,business_wise_discount_id=i,product_discount_ended_by_id=1,product_discount_started_by_id=18292)
    

/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField BusinessWiseProductDiscountTrace.start_date received a naive datetime (2022-12-13 06:02:17.137552) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField BusinessWiseProductDiscountTrace.end_date received a naive datetime (2051-05-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [19]:
product_34=list(BusinessWiseProductDiscount.objects.filter(discounted_price=60,product_id=34).values_list('id',flat=True))

In [20]:
import datetime
date = datetime.datetime.now()
for i in product_34:
    BusinessWiseProductDiscountTrace.objects.create(start_date=date,end_date='2051-05-01',mrp=60,discounted_price=60,business_wise_discount_id=i,product_discount_ended_by_id=1,product_discount_started_by_id=18292)
    

/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField BusinessWiseProductDiscountTrace.start_date received a naive datetime (2022-12-13 06:02:23.408508) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [17]:
route = Route.objects.get(session_id=1,id=242)
route.temp_route_of_main_route.all()

<QuerySet [<RouteTempRouteMap: RouteTempRouteMap object>]>

In [2]:
product_33=list(ICustomerTypeWiseProductDiscount.objects.filter(discounted_price=30,product_id=33).values_list('id',flat=True))

In [3]:
import datetime
date = datetime.datetime.now()
for i in product_33:
    ICustomerTypeWiseProductDiscountTrace.objects.create(start_date=date,end_date='2051-05-01',mrp=30,discounted_price=30,icustomer_type_wiser_product_discount_id=i,product_discount_ended_by_id=1,product_discount_started_by_id=1)
    

/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ICustomerTypeWiseProductDiscountTrace.start_date received a naive datetime (2022-12-13 12:34:37.628727) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ICustomerTypeWiseProductDiscountTrace.end_date received a naive datetime (2051-05-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [5]:
product_34=list(ICustomerTypeWiseProductDiscount.objects.filter(discounted_price=60,product_id=34).values_list('id',flat=True))

In [6]:
import datetime
date = datetime.datetime.now()
for i in product_34:
    ICustomerTypeWiseProductDiscountTrace.objects.create(start_date=date,end_date='2051-05-01',mrp=60,discounted_price=60,icustomer_type_wiser_product_discount_id=i,product_discount_ended_by_id=1,product_discount_started_by_id=1)
    

/home/selvi/Software/virtualenv/aavin_new/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ICustomerTypeWiseProductDiscountTrace.start_date received a naive datetime (2022-12-13 12:36:33.158941) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
